<a href="https://colab.research.google.com/github/guimalcantara/-Estrategia_Trading_NLP_e_Analise_De_Sentimento/blob/main/Estrategia_Trading_NLP_e_Analise_De_Sentimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP e Estratégia de Negociação Baseada em Análise de Sentimento


> Adicionar aspas



Neste estudo de caso, usamos NLP para construir uma estratégia de negociação combinando alguns dos conceitos que abordamos em alguns dos capítulos anteriores.




<a id='1'></a>
# 2. Introdução - Carregando os dados e pacotes python


<a id='2.1'></a>
## 2.1. Carregando os pacotes python

Como primeiro passo, verificamos se os pacotes adicionais necessários estão presentes, caso contrário, instalamos. Eles são verificados separadamente, pois não estão incluídos em requirement.txt, pois não são usados para todos os estudos de caso.

In [171]:
import pkg_resources
import pip
installedPackages = {pkg.key for pkg in pkg_resources.working_set}
required = {'nltk', 'spacy', 'textblob', 'backtrader'}
missing = required - installedPackages
if missing:
    !pip install --upgrade pip #The line should be indented
    !pip install nltk==3.9.1  #The line should be indented
    !pip install textblob==0.17.1 #The line should be indented
    !pip install spacy==3.7.2 #The line should be indented
    !pip install backtrader==1.9.74.123 #The line should be indented
    !pip install tensorflow
    !pip install keras
    !pip install yfinance --upgrade

In [ ]:
# Baixa os modelos de linguagem SpaCy necessários para NLP
!python -m spacy download pt_core_news_lg
!python -m spacy download pt_core_news_sm
!python -m spacy download en_core_web_lg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 20.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 38.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [ ]:

# DOWNLOADS E CONFIGURAÇÃO DO AMBIENTE NLP

import nltk
import spacy
import nltk.data
from textblob import TextBlob

# Download de recursos do NLTK
nltk.download('punkt')
nltk.download('vader_lexicon')

# Dicionário para armazenar os modelos carregados
spacy_models = {}

# Função segura para carregar modelos do spaCy
def carregar_modelo_spacy(nome_modelo):
    try:
        return spacy.load(nome_modelo)
    except OSError:
        print(f"Modelo {nome_modelo} não encontrado. Instale com:")
        print(f"!python -m spacy download {nome_modelo}")
        return None

# Carrega os modelos em português
spacy_models["pt"] = carregar_modelo_spacy("pt_core_news_lg") or carregar_modelo_spacy("pt_core_news_sm")

# Carrega o modelo em inglês (obrigatório para headlines internacionais)
spacy_models["en"] = carregar_modelo_spacy("en_core_web_lg")

# Verifica se os modelos foram carregados corretamente
assert spacy_models["pt"] is not None, "Modelo SpaCy para português não carregado."
assert spacy_models["en"] is not None, "Modelo SpaCy para inglês não carregado."

# ========================================
# TESTES DE FUNCIONAMENTO DOS MODELOS
# ========================================

# Testa com uma frase em português
doc_pt = spacy_models["pt"]("Esta é uma frase de exemplo.")
print("Modelo PT:", [(w.text, w.pos_) for w in doc_pt])

# Testa com uma frase em inglês
doc_en = spacy_models["en"]("This is a sample sentence.")
print("Modelo EN:", [(w.text, w.pos_) for w in doc_en])


Vamos carregar as bibliotecas

As bibliotecas **`scikit-learn`** (como `MLPClassifier`, `RandomForestClassifier`, `LogisticRegression`, etc.) são compatíveis com qualquer idioma, incluindo o **português brasileiro**. Isso ocorre porque os modelos de aprendizado de máquina da `scikit-learn` não são sensíveis ao idioma, mas sim às características dos dados (números, textos, etc.).

### Como funciona a compatibilidade com o português:

1. **Modelos de classificação**:

   * Os classificadores como **`MLPClassifier`**, **`RandomForestClassifier`**, **`LogisticRegression`**, etc., funcionam com **qualquer tipo de dado**, incluindo textos em português. No caso de textos, o que importa são as **features (características)** extraídas do texto, como a contagem de palavras, a frequência de termos, ou a presença de palavras específicas.
   * O modelo não "entende" diretamente o idioma, mas trabalha com **representações numéricas dos dados**, como as que você extrai ao usar técnicas como **TF-IDF** ou **Word Embeddings** (como **Word2Vec**, **GloVe**, ou **FastText**).

2. **Extração de características do texto**:
   Para usar esses classificadores com texto em português, é necessário representar as palavras ou frases em uma forma que o modelo consiga entender. As técnicas comuns incluem:

   * **Tokenização**: Dividir o texto em palavras ou unidades menores.
   * **Vetorização**: Transformar essas palavras em números, utilizando métodos como:

     * **Contagem de palavras** (`CountVectorizer`)
     * **TF-IDF** (`TfidfVectorizer`)
     * **Word Embeddings** (como **Word2Vec**, **GloVe**, ou **FastText**).

3. **Análise de Sentimentos**:

   * Quando você usa **análise de sentimentos** ou **classificação de texto** com essas bibliotecas, você geralmente treina o modelo com **exemplos de textos rotulados** (por exemplo, tweets ou resenhas em português). O modelo aprenderá padrões, mas o treinamento depende de como você prepara os dados de entrada.

4. **Resultados e Avaliação**:

   * As métricas como **`classification_report`**, **`confusion_matrix`**, e **`accuracy_score`** também são independentes do idioma. Elas avaliam a performance do modelo com base nos dados de entrada (no caso, textos em português) e as classes preditivas.
   * Essas métricas funcionam para qualquer conjunto de dados, independentemente de estar em inglês, português ou outro idioma.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Conjunto de manchetes sintéticas simulando notícias financeiras
texts = [
    "Ibovespa sobe com otimismo sobre corte de juros",
    "Dólar recua após anúncio do Banco Central",
    "Petrobras tem lucro recorde no segundo trimestre",
    "Mercado reage mal à nova política fiscal",
    "Ações da Vale despencam após queda no minério"
]

# Criando um Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# Convertendo os textos para sequências de números
sequences = tokenizer.texts_to_sequences(texts)
print("Sequências:", sequences)

# Visualizando o vocabulário gerado pelo Tokenizer
print("Vocabulário:", tokenizer.word_index)



In [ ]:
# Bibliotecas para Processamento de Linguagem Natural (PLN)
from textblob import TextBlob
import spacy
import nltk
import warnings
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')  # Baixa o léxico necessário para análise de sentimentos
import csv
import pandas as pd

# Carregue o modelo de linguagem
import pt_core_news_lg
nlp = pt_core_news_lg.load()
import en_core_web_lg
nlp_en = en_core_web_lg.load()
# Bibliotecas para processar manchetes de notícias
from lxml import etree
import json
from io import StringIO
from os import listdir
from os.path import isfile, join
from pandas.tseries.offsets import BDay  # Offset de negócios (útil para datas)
from scipy.stats.mstats import winsorize  # Método de Winsorização (para lidar com valores extremos)
from copy import copy  # Criação de cópias superficiais de objetos

# Bibliotecas para classificação e modelagem de sentimentos
from sklearn.neural_network import MLPClassifier  # Classificador de redes neurais
from sklearn.ensemble import RandomForestClassifier  # Classificador de floresta aleatória
from sklearn.linear_model import LogisticRegression  # Regressão logística
from sklearn.tree import DecisionTreeClassifier  # Classificador de árvore de decisão
from sklearn.neighbors import KNeighborsClassifier  # K-vizinhos mais próximos
from sklearn.svm import SVC  # Máquinas de vetores de suporte (SVM)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  # Métricas de avaliação

# Bibliotecas para modelagem de sentimento com redes neurais (Deep Learning)
#from keras.preprocessing.text import Tokenizer  # Tokenização de texto para redes neurais - Commented out keras import
from tensorflow.keras.preprocessing.text import Tokenizer  # Usando tensorflow.keras ao invés de keras
from tensorflow.keras.preprocessing.sequence import pad_sequences  # Preenchimento de sequências de texto
from tensorflow.keras.models import Sequential  # Modelo sequencial de redes neurais
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding  # Camadas de redes neurais, incluindo a camada de embedding
from tensorflow.keras.layers import Embedding  # Camada de embedding para palavras

# Bibliotecas para análise estatística e visualização
import statsmodels.api as sm  # Análise estatística
import seaborn as sns  # Visualização de dados
import pandas as pd  # Manipulação de dados
import numpy as np  # Funções matemáticas e de álgebra linear
import datetime  # Manipulação de datas e horas
from datetime import date  # Trabalhando com datas
import matplotlib.pyplot as plt  # Visualização de gráficos
import yfinance as yf  # Para obter dados financeiros de ações

# Bibliotecas adicionais
import json  # Manipulação de dados em formato JSON
import zipfile  # Trabalhar com arquivos ZIP
import os.path  # Manipulação de caminhos de arquivos
import sys  # Funções do sistema operacional



In [ ]:
#Diable the warnings
import warnings
warnings.filterwarnings('ignore')

<a id='2.2'></a>
## 2.2. Carregando os dados de preços das ações

Os dados de preços das ações são carregados nesta etapa do Yahoo Finance. Os dados carregados são salvos em csv para uso posterior.

In [ ]:
import yfinance as yf
import pandas as pd
import os

# Mapeamento manual de dias da semana em português
dias_semana_pt = {
    'Monday': 'Segunda-feira',
    'Tuesday': 'Terça-feira',
    'Wednesday': 'Quarta-feira',
    'Thursday': 'Quinta-feira',
    'Friday': 'Sexta-feira',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

# Lista de tickers brasileiros
tickers = ['VALE3.SA', 'PETR4.SA', 'ITUB4.SA', 'BBAS3.SA']

# Período de análise
start = '2022-01-01'
end = '2024-12-31'

# Lista para acumular DataFrames
dados_completos = []

for ticker in tickers:
    ticker_yf = yf.Ticker(ticker)
    dados = ticker_yf.history(start=start, end=end)

    # Garante que o índice seja convertido em coluna
    dados = dados.reset_index()

    # Criação das colunas formatadas
    dados['Data'] = dados['Date'].dt.date.astype(str)
    dados['Hora'] = dados['Date'].dt.time.astype(str)
    dados['Dia_da_semana'] = dados['Date'].dt.day_name().map(dias_semana_pt)

    dados['Ticker'] = ticker

    # Seleciona e reordena colunas
    dados_formatado = dados[[
        'Data', 'Dia_da_semana',
        'Open', 'High', 'Low', 'Close',
        'Volume', 'Dividends', 'Stock Splits', 'Ticker'
    ]]

    dados_completos.append(dados_formatado)

# Concatenação final
df_final = pd.concat(dados_completos, ignore_index=True)

# Criação de diretório
os.makedirs("data", exist_ok=True)

# Caminhos dos arquivos
caminho_csv = "data/ReturnData.csv"
caminho_excel = "data/ReturnData.xlsx"

# Salvamento dos arquivos
df_final.to_csv(caminho_csv, sep='|', index=False)
df_final.to_excel(caminho_excel, index=False, engine='openpyxl')

# Exibição interativa (Google Colab)
from google.colab import data_table
data_table.enable_dataframe_formatter()
data_table.DataTable(df_final)

Os dados contêm os tickers e seus retornos. Na próxima etapa, limpamos os dados e garantimos que o ponto de partida seja 2022 e que os NAs nos dados sejam removidos.

<a id='3'></a>

# 3. Preparação dos Dados

Dividimos a preparação dos dados em algumas etapas da seguinte forma:
* Carregando e pré-processando os dados das notícias
* Preparando os dados combinados

<a id='3.1'></a>

## 3.1  Carregando e pré-processando os dados das notícias

Os dados das notícias são baixados do feed de notícias RSS e o arquivo é baixado no formato json e os arquivos json para diferentes datas são mantidos em uma pasta compactada.

## 3.1.1  Coleta Automatizada de Notícias Financeiras via RSS
## Coleta Automatizada de Notícias Financeiras via RSS

O script desenvolvido tem como objetivo central coletar notícias publicadas na internet relacionadas a ações brasileiras listadas na B3, utilizando a API RSS do Google News como canal de extração. Para tanto, emprega uma arquitetura em Python composta por módulos consagrados como `requests`, `pandas`, `BeautifulSoup` e `dateutil`, integrando técnicas de *web scraping*, manipulação de dados e limpeza textual.

### 1. Definição do Escopo Temporal e Semântico

Inicialmente, define-se um dicionário com os termos de busca associados a cada **ticker** de interesse (como `'VALE3.SA'`), permitindo múltiplas variações nomenclaturais relevantes para mecanismos de busca. O intervalo de coleta abrange um período fixo entre 1º de janeiro de 2022 e 31 de dezembro de 2024.

A seguir, o código gera **janelas temporais de 6 dias** consecutivos para modular a busca. Esta divisão busca contornar limitações de resposta dos serviços RSS e assegurar granularidade temporal adequada para análise posterior.

### 2. Consulta ao Google News RSS

Para cada janela de tempo e para cada ticker, é construída uma URL parametrizada com uma *query* em linguagem booleana — incluindo todos os termos relacionados ao ativo — e delimitadores temporais `after` e `before`. A requisição HTTP é feita com cabeçalho que simula um navegador, e os resultados são processados com o **BeautifulSoup**, que permite extrair título, descrição, data, horário e URL de cada notícia.

Além disso, a biblioteca `langdetect` é utilizada para inferir o idioma da manchete, o que é uma prática comum para filtrar ruídos em bases multilinguísticas. Todo esse processo é encapsulado em uma função robusta com tratamento de exceções para garantir continuidade da execução mesmo em casos de falha de rede ou estrutura HTML malformada.

### 3. Estruturação dos Dados

As notícias coletadas são armazenadas como dicionários em uma lista principal. Ao final, esse material é convertido em um `DataFrame` do pandas para facilitar operações de tratamento e análise.

#### Deduplicação e Limpeza

A deduplicação se dá com base em manchetes normalizadas (`headline_clean`), onde caracteres não alfanuméricos são removidos e o texto é transformado em minúsculas. A contagem de repetições é calculada por data e título, permitindo a ordenação por relevância.

### 4. Exportação dos Dados

Os dados únicos, organizados e limpos, são exportados em formato JSON, com codificação UTF-8 preservada (`force_ascii=False`) e estrutura orientada a registros. O arquivo final é salvo localmente na pasta `dados`.

### 5. Feedback ao Usuário

A aplicação fornece um resumo estatístico ao final da execução, indicando o número total de notícias únicas coletadas, o caminho do arquivo salvo e a distribuição de notícias por ticker.

---

## Considerações Técnicas

A abordagem é eficiente, modular e robusta. Utiliza técnicas idiomáticas de Python e boas práticas como:

* Modularização por funções;
* Uso de `try/except` para tolerância a falhas;
* Normalização textual para comparação semântica;
* Aplicação de `tqdm` para *feedback visual* com barra de progresso;
* Estrutura de dados bem organizada para exportação e uso posterior em análises de NLP ou modelagem financeira.

---



Coleta de notícias

In [ ]:
!pip install -q beautifulsoup4 pandas python-dateutil langdetect tqdm

In [ ]:
# ===============================================================
# IMPORTAÇÃO DE BIBLIOTECAS NECESSÁRIAS
# ===============================================================

import os
import json
import zipfile
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta

# ===============================================================
# DEFINIÇÃO DOS TICKERS E TERMOS DE BUSCA ASSOCIADOS
# ===============================================================

termos_de_busca = {
    'VALE3.SA': ['VALE3', 'VALE3.SA', 'BVMF:VALE3'],
    'PETR4.SA': ['PETR4', 'PETR4.SA', 'BVMF:PETR4'],
    'ITUB4.SA': ['ITUB4', 'ITUB4.SA', 'BVMF:ITUB4'],
    'BBAS3.SA': ['BBAS3', 'BBAS3.SA', 'BVMF:BBAS3']
}

# Intervalo de datas para busca de notícias
inicio_coleta = datetime.strptime('2022-01-01', '%Y-%m-%d')
fim_coleta    = datetime.strptime('2024-12-31', '%Y-%m-%d')

# ===============================================================
# FUNÇÃO PARA GERAR INTERVALOS MENSAIS ENTRE DUAS DATAS
# ===============================================================

def gerar_periodos_mensais(start, end):
    periodos = []
    atual = start
    while atual < end:
        fim_mes = atual + relativedelta(months=1) - relativedelta(days=1)
        if fim_mes > end:
            fim_mes = end
        periodos.append((atual, fim_mes))
        atual = fim_mes + relativedelta(days=1)
    return periodos

# ===============================================================
# FUNÇÃO PARA COLETAR NOTÍCIAS VIA GOOGLE NEWS RSS
# ===============================================================

def google_news_rss_range(ticker, termos, data_inicio, data_fim):
    # Monta query de busca com OR entre termos
    query = "(" + " OR ".join([f'"{t}"' if ' ' in t else t for t in termos]) + ")"

    # Monta a URL do RSS com filtros de data
    url = (
        'https://news.google.com/rss/search'
        f'?q={query}+after:{data_inicio.strftime("%Y-%m-%d")}+before:{data_fim.strftime("%Y-%m-%d")}'
        '&hl=pt-BR&gl=BR&ceid=BR:pt-419'
    )

    noticias = []

    try:
        # Requisição HTTP com cabeçalho do navegador
        resposta = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=10)
        soup = BeautifulSoup(resposta.content, 'xml')

        # Percorre cada item do RSS
        for item in soup.find_all('item'):
            pub_date = datetime.strptime(item.pubDate.text, '%a, %d %b %Y %H:%M:%S %Z')

            # Filtra por data
            if not (data_inicio <= pub_date <= data_fim):
                continue

            titulo  = item.title.text
            link    = item.link.text
            resumo  = BeautifulSoup(item.description.text, 'html.parser').get_text()

            # Valida se algum termo está presente no título ou link
            if any(term.lower() in (titulo + link).lower() for term in termos):
                noticias.append({
                    'ticker': ticker,
                    'date': pub_date.strftime('%Y-%m-%d'),
                    'time': pub_date.strftime('%H:%M'),
                    'headline': titulo,
                    'summary': resumo,
                    'source': 'Google News RSS',
                    'url': link,
                    'language': 'pt',
                    'scraped_at': datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
                })

        return noticias

    except Exception as e:
        print(f"[ERRO] Falha ao buscar RSS para {ticker} ({data_inicio} a {data_fim}): {e}")
        return []

# ===============================================================
# EXECUÇÃO DA COLETA E ARMAZENAMENTO EM ARQUIVOS JSON
# ===============================================================

# Criação do diretório de saída
os.makedirs("noticias_json", exist_ok=True)

# Geração dos períodos mensais
periodos = gerar_periodos_mensais(inicio_coleta, fim_coleta)

# Laço principal para cada ticker
for ticker, termos in termos_de_busca.items():
    todas_noticias = []

    for data_ini, data_fim in periodos:
        lote = google_news_rss_range(ticker, termos, data_ini, data_fim)
        todas_noticias.extend(lote)

    # Salvamento em arquivo JSON por ticker
    if todas_noticias:
        caminho_arquivo = os.path.join("noticias_json", f"{ticker}.json")
        with open(caminho_arquivo, "w", encoding="utf-8") as f:
            json.dump(todas_noticias, f, indent=2, ensure_ascii=False)
        print(f"[OK] Notícias salvas para {ticker} ({len(todas_noticias)} registros)")

# ===============================================================
# COMPACTAÇÃO DOS ARQUIVOS EM UM ARQUIVO ZIP
# ===============================================================

with zipfile.ZipFile("noticias_tickers.zip", "w", zipfile.ZIP_DEFLATED) as zipf:
    for nome_arquivo in os.listdir("noticias_json"):
        caminho = os.path.join("noticias_json", nome_arquivo)
        zipf.write(caminho, arcname=nome_arquivo)

print("\n✅ Arquivo ZIP gerado com sucesso: noticias_tickers.zip")


In [ ]:
import os
import json
import zipfile
import pandas as pd

# ===============================================================
# 1. EXTRAÇÃO DOS ARQUIVOS DO ZIP (GOOGLE COLAB: /content)
# ===============================================================

# Caminho do ZIP no ambiente Colab
caminho_zip = "/content/noticias_tickers.zip"
pasta_extracao = "/content/noticias_json"
os.makedirs(pasta_extracao, exist_ok=True)

# Extração dos arquivos JSON
with zipfile.ZipFile(caminho_zip, "r") as zipf:
    zipf.extractall(pasta_extracao)

print(" Arquivos extraídos para pasta '/content/noticias_json/'")

# ===============================================================
# 2. CONSOLIDAÇÃO, DEDUPLICAÇÃO E ANOTAÇÃO DE REPETIÇÕES
# ===============================================================

# Criação do diretório de saída
pasta_saida = "/content/dados"
os.makedirs(pasta_saida, exist_ok=True)

# Carregar todos os arquivos JSON extraídos
todas_noticias = []
for nome_arquivo in os.listdir(pasta_extracao):
    if nome_arquivo.endswith(".json"):
        caminho = os.path.join(pasta_extracao, nome_arquivo)
        with open(caminho, "r", encoding="utf-8") as f:
            noticias = json.load(f)
            todas_noticias.extend(noticias)

# Conversão para DataFrame
df = pd.DataFrame(todas_noticias)

# Remoção de duplicatas com base em headline e summary
df_deduplicado = df.drop_duplicates(subset=['headline', 'summary'])

# Contagem de repetições
reps = df.groupby(['headline', 'summary']).size().reset_index(name='repetitions')

# Junção com coluna repetitions
df_final = pd.merge(df_deduplicado, reps, on=['headline', 'summary'], how='left')

# Reorganização de colunas (opcional)
colunas = ['date', 'time', 'ticker', 'headline', 'summary', 'repetitions',
           'source', 'url', 'language', 'scraped_at']
df_final = df_final[colunas]

# Salvamento do JSON final consolidado
saida = os.path.join(pasta_saida, "noticias_ativos_brasileiros.json")
df_final.to_json(saida, orient='records', indent=2, force_ascii=False)

# Resumo
print(f"\n Arquivo consolidado salvo: {saida}")
print(f" Total de notícias únicas: {len(df_final)}")


### Vamos ver o conteúdo do arquivo json
Nesta etapa, realiza-se a leitura de um arquivo no formato JSON estruturado contendo notícias financeiras previamente coletadas via Google News RSS. O objetivo é carregar o conteúdo para análise exploratória e manipulação dentro do ambiente Python, utilizando a biblioteca `pandas` .

In [ ]:
# --- Carregando arquivo .json consolidado ---
import pandas as pd
import json
from datetime import datetime

# Caminho do arquivo gerado na etapa anterior
caminho_json = "/content/dados/noticias_ativos_brasileiros.json"

# Carregamento e Leitura
with open(caminho_json, "r", encoding="utf-8") as f:
    noticias_raw = json.load(f)
# Mostra o primeiro dicionário (notícia)
df_news = pd.read_json(caminho_json, convert_dates=["date", "scraped_at"])

# Pré-visualização da estrutura bruta
noticias_raw[0]
# Visualização das primeiras entradas
df_news.head(3)


### Conversão para `DataFrame` estruturado



In [ ]:
# Converte a coluna 'date' para tipo datetime.date (sem hora)
df_news['date'] = df_news['date'].dt.date

# Padroniza as colunas
df_news = df_news[[
    'ticker', 'date', 'time', 'headline', 'summary', 'source', 'url', 'language', 'scraped_at'
]]

# Remove entradas sem headline ou data
df_news.dropna(subset=['headline', 'date'], inplace=True)

# Verifica resultado
print(f"Número total de notícias válidas: {len(df_news)}")
df_news.sample(3)

In [ ]:
# --- Carregando arquivo .json consolidado ---
import pandas as pd
import json
import google.colab.data_table as data_table
from datetime import datetime

# Caminho do arquivo gerado na etapa anterior
caminho_json = "dados/noticias_ativos_brasileiros.json"

# Carregamento e Leitura
with open(caminho_json, "r", encoding="utf-8") as f:
    noticias_raw = json.load(f)
# Mostra o primeiro dicionário (notícia)
df_news = pd.read_json(caminho_json, convert_dates=["date", "scraped_at"])

# Pré-visualização da estrutura bruta
noticias_raw[0]
# Visualização das primeiras entradas
df_news.head(3)

# Converte a coluna 'date' para tipo datetime.date (sem hora)
df_news['date'] = df_news['date'].dt.date

# Padroniza as colunas
df_news = df_news[[
    'ticker', 'date', 'time', 'headline', 'summary', 'source', 'url', 'language', 'scraped_at'
]]

# Remove entradas sem headline ou data
df_news.dropna(subset=['headline', 'date'], inplace=True)

# Verifica resultado
print(f"Número total de notícias válidas: {len(df_news)}")
df_news.sample(3)

# Assign df_news to data_df_news
data_df_news = df_news  # This line is crucial to define data_df_news

# Visualizar data_df_news (manchetes + tickers + datas)
print(" Visualizando data_df_news (notícias):")
print(data_df_news.dtypes)
print(data_df_news.head(5))
print(f"\nTotal de registros: {len(data_df_news)}")
print(data_df_news['ticker'].value_counts())
data_table.DataTable(data_df_news)  # Assuming data_table is already imported

<a id='3.2'></a>

## 3.2 Preparando os dados combinados

Nesta etapa, extraímos o retorno de evento, que é o retorno que corresponde ao evento. Fazemos isso porque, às vezes, as notícias são reportadas tardiamente e, outras vezes, são reportadas após o fechamento do mercado. Ter uma janela um pouco mais ampla garante que capturemos a essência do evento. O retorno de evento é definido da seguinte forma: $ R_{t-1} + R_t + R_{t+1} $

Onde, $ R_{t-1} $, $ R_{t+1} $ são o retorno antes e depois dos dados da notícia e $ R_{t} $  é o retorno no dia da notícia (ou seja, tempo $t$ )




In [ ]:
!pip install openpyxl --quiet

In [ ]:
# Etapa 3.2.1 — Preparação dos Dados Combinados

import pandas as pd

# Leitura do arquivo Excel
df_ticker_return = pd.read_excel("/content/data/ReturnData.xlsx")

# --- Cálculo dos retornos a partir dos dados ajustados ---
# Calcula o retorno diário com base na coluna 'Close'
df_ticker_return['ret_curr'] = df_ticker_return['Close'].pct_change()

# Calcula o retorno do evento (soma de Rt-1 + Rt + Rt+1)
df_ticker_return['eventRet'] = (
    df_ticker_return['ret_curr'] +
    df_ticker_return['ret_curr'].shift(-1) +
    df_ticker_return['ret_curr'].shift(1)
)

# Reset do índice (assumido necessário apenas se 'Data' era índice — neste caso, por segurança)
df_ticker_return.reset_index(drop=True, inplace=True)

# Converte a coluna 'Data' para datetime.date (sem hora)
df_ticker_return['date'] = pd.to_datetime(df_ticker_return['Data']).apply(lambda x: x.date())

# Renomeia a coluna de ticker, se necessário, para garantir compatibilidade com os dados de notícias
df_ticker_return.rename(columns={'Ticker': 'ticker'}, inplace=True)

# Verificação rápida
print("\n Retornos calculados com sucesso:")
print(df_ticker_return[['Data', 'ticker', 'Close', 'ret_curr', 'eventRet', 'date']].head())



Agora que temos todos os dados preparados, prepararemos um dataframe combinado que terá as manchetes das notícias mapeadas para a data, Retorno de Evento e ticker da ação. Este dataframe será usado para análise posterior para o modelo de análise de sentimento e para construir a estratégia de negociação (trading).

In [ ]:
# Junção com Dados de Notícias
# --- Leitura das notícias salvas ---
noticias_json = "dados/noticias_ativos_brasileiros.json"
data_df_news = pd.read_json(noticias_json)
data_df_news['date'] = pd.to_datetime(data_df_news['date']).dt.date
data_df_news = data_df_news[['ticker', 'headline', 'date']]  # apenas o necessário

# --- Junção com dados de retorno ---
combinedDataFrame = pd.merge(
    data_df_news,
    df_ticker_return,
    how='left',
    left_on=['ticker', 'date'],
    right_on=['ticker', 'date'] # Changed 'Ticker' to 'ticker' in right_on
)

# --- Seleção final de colunas e limpeza ---
data_df = combinedDataFrame[['ticker', 'headline', 'date', 'eventRet', 'Close']]
data_df = data_df.dropna().reset_index(drop=True)

# --- Visualização inicial ---
data_df.head(3)

Vamos salvar os dados em um arquivo csv para serem usados posteriormente, para que a etapa de processamento de dados possa ser pulada toda vez que formos realizar a análise.

In [ ]:
# Exportação Final para CSV

# --- Salvamento do dataset final ---
import os
os.makedirs("dados", exist_ok=True)
caminho_saida = "/content/AnaliseSentimentoNLP/AnaliseSentimentoNLP/Step3_NewsAndReturnData.csv"
data_df.to_csv(caminho_saida, sep='|', index=False)

# --- Resumo da operação ---
print(f"\n Arquivo final salvo em: {caminho_saida}")
print(f"Total de registros combinados: {len(data_df)}")
print(data_df.groupby('ticker').size())


In [ ]:
data_df.dropna().to_csv(r'Data\Step3_NewsAndReturnData.csv', sep='|', index=False)

<a id='3.3'></a>
## 3.3 Carregando os dados pré-processados
#### Comece a partir desta etapa caso você não queira executar as etapas anteriores de pré-processamento.

In [ ]:
# --- Etapa 3.3: Recarregando dados diretamente do CSV (caso queira pular a etapa de preparação) ---
print("\n Recarregando dados salvos (para uso direto em análise):")

#caminho_saida is the variable where the csv was saved in the previous cell
data_df = pd.read_csv(caminho_saida, sep='|') #Change caminho_csv to caminho_saida
data_df = data_df.dropna()

# --- Verificação básica ---
print(" Estrutura dos dados carregados:")
print(data_df.dtypes)
print(data_df.head(3))
print(f"\n Total de registros carregados: {data_df.shape[0]}")
print(f" Total de tickers únicos: {data_df['ticker'].nunique()}")
print(f" Lista de tickers: {list(data_df['ticker'].unique())}")

In [ ]:
print(data_df.shape, data_df.ticker.unique().shape)

Nesta etapa, preparamos um dataframe limpo que com:

* **2607 registros** (linhas de manchetes combinadas com dados de retorno),
* **5 colunas**: `'ticker'`, `'headline'`, `'date'`, `'eventRet'`, `'Close'`,
* **4 tickers únicos**, o que confirma que os dados foram combinados para todas as ações: `'VALE3.SA'`, `'PETR4.SA'`, `'ITUB4.SA'`, `'BBAS3.SA'`.

<a id='4'></a>
# 4. Avaliar Modelos para Análise de Sentimento

Nesta seção, abordaremos as três abordagens diferentes a seguir para obter os sentimentos das notícias, que usaremos para construir a estratégia de negociação (trading).

* Modelo predefinido - pacote TextBlob
* Modelo Ajustado - Algoritmos de Classificação e LSTM
* Modelo baseado em léxico financeiro

Também exploraremos a diferença entre as diferentes formas de realizar a análise de sentimento. Vamos seguir os passos.


<a id='4.1'></a>
## 4.1 - Aplicação e Comparação de Modelos de Sentimento em Português

Nesta etapa, serão aplicados modelos de linguagem natural pré-treinados, com foco em análise de sentimentos em português, sobre os títulos de notícias financeiras contidos na coluna `headline` do DataFrame `data_df`. O objetivo principal consiste na obtenção de **pontuações contínuas de polaridade**, no intervalo $[-1, +1]$, que serão posteriormente confrontadas com os **retornos de evento** (`eventRet`). A finalidade é investigar potenciais correlações entre o conteúdo semântico dos títulos e o comportamento do mercado.

### Modelos Utilizados

| Modelo | Nome no Hugging Face                           | Tipo       | Idioma    | Saída original                    | Mapeamento para escore contínuo   |
| ------ | ---------------------------------------------- | ---------- | --------- | --------------------------------- | --------------------------------- |
| A      | `lucas-leme/FinBERT-PT-BR`                     | Financeiro | Português | `POSITIVE`, `NEGATIVE`, `NEUTRAL` | `-1` (NEG), `0` (NEU), `+1` (POS) |
| B      | `sptech-ai/sptech.template.ai.model.sentiment` | Genérico   | Português | `POSITIVE`, `NEGATIVE`            | `-1` (NEG), `+1` (POS)            |
| C      | `turing-usp/FinBertPTBR`                       | Financeiro | Português | `POSITIVE`, `NEGATIVE`, `NEUTRAL` | `-1` (NEG), `0` (NEU), `+1` (POS) |

### Conversão de Saídas Categóricas para Polaridade Contínua

Para garantir a compatibilidade com métodos quantitativos e análises estatísticas, as classificações categóricas dos modelos serão transformadas conforme o seguinte esquema de codificação:

* `NEGATIVE` → `-1`
* `NEUTRAL` → `0`
* `POSITIVE` → `+1`

Nos casos em que o modelo fornece **distribuições de probabilidade sobre as classes** (a partir dos *logits*), será adotada uma abordagem probabilística para o cálculo do escore contínuo, conforme a equação:

$$
\text{Polaridade} = P(\text{POSITIVE}) \cdot (+1) + P(\text{NEUTRAL}) \cdot 0 + P(\text{NEGATIVE}) \cdot (-1)
$$

Essa formulação permite capturar com maior fidelidade a incerteza do modelo, oferecendo um resultado mais informativo e expressivo, análogo ao funcionamento da biblioteca `TextBlob`.

### Considerações Técnicas

Os modelos utilizados não fornecem, de forma nativa, uma métrica de polaridade contínua. No entanto, a conversão dos *logits* para probabilidades por meio da função `softmax` permite a emulação confiável desse comportamento. Tal estratégia é altamente recomendada para aplicações em finanças quantitativas, nas quais a granularidade e a robustez dos dados de entrada são fatores determinantes para a qualidade das inferências e decisões baseadas em modelos.

###  Rodar a análise pra esses modelos agora é só por curiosidade mesmo, só pra ver se o ajuste funciona.

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm.notebook import tqdm

# Dispositivo de execução
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Token de acesso ao Hugging Face (pré-configurado em variável de ambiente)
hf_token = os.environ.get("HF_TOKEN")

def calcular_polaridade_continua(
    model_id: str,
    n_classes: int,
    textos: pd.Series
) -> pd.Series:
    """Retorna escore contínuo: P_POS*1 + P_NEU*0 + P_NEG*(-1)."""
    tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=hf_token)
    model = (AutoModelForSequenceClassification
             .from_pretrained(model_id, use_auth_token=hf_token)
             .to(device))
    model.eval()

    def pontuar(texto: str) -> float:
        inputs = tokenizer(
            texto,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=512
        ).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
        probs = F.softmax(logits, dim=1).squeeze().cpu().numpy()
        if n_classes == 3:
            # ordem: [NEG, NEU, POS]
            return float(probs[2]*1 + probs[1]*0 + probs[0]*-1)
        elif n_classes == 2:
            # ordem: [NEG, POS]
            return float(probs[1]*1 + probs[0]*-1)
        else:
            return np.nan

    return textos.progress_apply(pontuar)


In [ ]:
data_path = "/content/Data\Step3_NewsAndReturnData.csv"
if not os.path.exists(data_path):
    raise FileNotFoundError(f"Arquivo não encontrado em {data_path}")
data_df = pd.read_csv(data_path, sep="|")


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
tqdm.pandas()

def aplicar_modelo_sentimento(model_name, task='text-classification'):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    pipe = pipeline(task=task, model=model, tokenizer=tokenizer)
    return pipe


In [ ]:
import os
from google.colab import userdata

# Lê o token, nome e e-mail diretamente dos secrets
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')


### 1. Defina o dicionário de modelos

In [ ]:
modelos = {
    "FinBERT_PT_BR": {
        "repo": "lucas-leme/FinBERT-PT-BR",
        "n_classes": 3
    },
    "FinBERT_Turing": {
        "repo": "turing-usp/FinBertPTBR",
        "n_classes": 3
    },
    "Multilingual_Sentiment": {
        "repo": "tabularisai/multilingual-sentiment-analysis",
        "n_classes": 3
    }
}

for nome, cfg in modelos.items():
    try:
        print(f"\nAplicando modelo {nome}...")
        coluna = f"sentiment_{nome}"
        data_df[coluna] = calcular_polaridade_continua(
            model_id=cfg["repo"],
            n_classes=cfg["n_classes"],
            textos=data_df["headline"]
        )
    except Exception as e:
        print(f"Erro em {nome}: {e}")


3. Cálculo de Correlação com Retorno de Evento



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

# 1. Cálculo de correlação e p-valor
resultados = []
for nome_modelo in modelos.keys():
    coluna = f"sentiment_{nome_modelo}"
    if coluna in data_df.columns:
        x = data_df[coluna]
        y = data_df["eventRet"]
        corr, p_valor = stats.pearsonr(x, y)
        resultados.append({
            "Modelo": nome_modelo,
            "Correlação": corr,
            "p-valor": p_valor
        })
        print(f"{nome_modelo}: correlação = {corr:.4f}, p-valor = {p_valor:.4g}")

# 2. DataFrame de resultados
corr_df = pd.DataFrame(resultados).set_index("Modelo")

# 3. Gráfico de barras
plt.figure(figsize=(6,4))
corr_df["Correlação"].plot(kind="bar", color="skyblue", edgecolor="black")
plt.axhline(0, color="gray", linewidth=0.8)
plt.title("Coeficiente de Correlação entre Sentimento e eventRet")
plt.ylabel("Pearson r")
plt.tight_layout()
plt.show()

# 4. Gráficos de dispersão
for nome_modelo in modelos.keys():
    coluna = f"sentiment_{nome_modelo}"
    if coluna in data_df.columns:
        plt.figure(figsize=(5,4))
        plt.scatter(data_df[coluna], data_df["eventRet"], alpha=0.4, s=10)
        plt.title(f"Dispersão: {nome_modelo}")
        plt.xlabel("Pontuação de Sentimento")
        plt.ylabel("Retorno (eventRet)")
        plt.tight_layout()
        plt.show()


<a id='4.2'></a>
## 4.2 – Análise de Sentimento com Classificadores Supervisionados e LSTM


## Construção de Pipeline com LSTM para Classificação Binária

Com base no conjunto consolidado contendo as variáveis `headline` e `eventRet`, foi construída uma *pipeline* de classificação binária utilizando redes neurais recorrentes do tipo **LSTM (Long Short-Term Memory)**.

### Preparação dos Dados

* A variável `eventRet` foi transformada em uma variável alvo binária:

  * Valores **≥ 0** foram rotulados como **1** (*retorno positivo*);
  * Valores **< 0** foram rotulados como **0** (*retorno negativo*).

* As manchetes textuais (`headline`) foram convertidas em vetores semânticos utilizando técnicas de *word embeddings*.

* Foram aplicadas as seguintes etapas de pré-processamento:

  * **Tokenização**: conversão das palavras em índices numéricos;
  * **Padding**: preenchimento das sequências para padronização do comprimento.

### Arquitetura do Modelo

O modelo foi estruturado com as seguintes camadas:

* `Embedding`:

  * Dimensão vetorial: **128**

* `LSTM`:

  * Unidades: **64**
  * Aplicação de **dropout** para regularização.

* Camada densa final:

  * Ativação: **sigmoide**
  * Finalidade: produção de probabilidade binária (0 ou 1)

### Estratégia de Treinamento

* O conjunto de dados foi dividido da seguinte forma:

  * **70%** para treino;
  * **30%** para teste.

* Foram utilizadas técnicas de:

  * **Validação cruzada**;
  * **Early stopping**, a fim de evitar *overfitting*.

### Resultados Obtidos

* **Acurácia no conjunto de treino**: **96,6%**
* **Acurácia no conjunto de teste**: **91,0%**
* **Correlação entre as predições e `eventRet`**: **0,446**
* **Matriz de confusão**: evidenciou equilíbrio na classificação entre as classes.

### Pós-Treinamento

Após o treinamento:

* O modelo foi aplicado ao **conjunto completo** de dados do intervalo de 01 de janeiro de 2022 a 31 de dezembro de 2024.
* As **predições** foram armazenadas para:

  * Análise quantitativa posterior;
  * Integração com **simulações de estratégias financeiras**.




In [ ]:
!pip install --upgrade pip
!pip install keras tensorflow scikit-learn

In [ ]:
import numpy as np
import pandas as pd

# Parte 2.1 — Definição de sentiments_data
# logo após ter carregado data_df:
sentiments_data = data_df.copy()

# Agora a verificação prossegue sem NameError
if 'eventRet' not in sentiments_data.columns:
    if 'eventRet' in data_df.columns:
        data_df['date']        = pd.to_datetime(data_df['date'],        errors='coerce').dt.date
        sentiments_data['date']= pd.to_datetime(sentiments_data['date'],errors='coerce').dt.date
        sentiments_data = pd.merge(
            sentiments_data,
            data_df[['date','ticker','eventRet']],
            on=['date','ticker'],
            how='left'
        )
    else:
        raise KeyError("Coluna 'eventRet' não encontrada em sentiments_data nem em data_df")

In [ ]:
# Remove registros sem manchete ou retorno
df_modelo = sentiments_data.dropna(subset=["headline", "eventRet"]).copy()

# Cria variável binária: 1 se eventRet ≥ 0, senão 0
df_modelo["sentiment"] = df_modelo["eventRet"].apply(lambda x: 1 if x >= 0 else 0)

# Exibe balanceamento
print("Distribuição dos rótulos:")
print(df_modelo["sentiment"].value_counts())


In [ ]:
# Função de vetorização por idioma
def vetorizar_manchete(texto, idioma='pt'):
    modelo = spacy_models.get(idioma, spacy_models['pt'])
    doc = modelo(texto)
    return doc.vector

# Gera matriz de embeddings
print("Gerando embeddings das manchetes... isso pode demorar.")
X = np.vstack(df_modelo["headline"].apply(lambda x: vetorizar_manchete(str(x))).values)

# Define variável alvo y
y = df_modelo["sentiment"].values


In [ ]:
#Parte 5. Tokenização, sequência e padding para LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Configura tokenizador
vocabulary_size = 20000
tokenizer = Tokenizer(num_words=vocabulary_size)
tokenizer.fit_on_texts(df_modelo["headline"])

# Converte textos em sequências e aplica padding
sequences = tokenizer.texts_to_sequences(df_modelo["headline"])
X_seq = pad_sequences(sequences, maxlen=50, padding='post', truncating='post')

# Divide em treino (70 %) e teste (30 %)
X_train_LSTM, X_test_LSTM, y_train_LSTM, y_test_LSTM = train_test_split(
    X_seq,
    y,
    test_size=0.3,
    stratify=y,
    random_state=42
)


In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# KERNEL DEVE SER REINICIALIZADO ANTES DA EXECUÇÃO
# ──────────────────────────────────────────────────────────────────────────────

# 1. VARIÁVEIS DE AMBIENTE (antes de importar TensorFlow)
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
# os.environ["CUDA_VISIBLE_DEVICES"] = ""  # força CPU, se necessário

# 2. IMPORTS
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import mixed_precision
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 3. LIMPEZA DE SESSÃO
K.clear_session()

# 4. PRECISÃO MISTA
mixed_precision.set_global_policy('mixed_float16')

# 5. HYPERPARÂMETROS
VOCAB_SIZE   = 10000
SEQ_LENGTH   = 50
EMBED_DIM    = 64
RNN_UNITS    = 16
BATCH_SIZE   = 64
EPOCHS       = 10
VAL_SPLIT    = 0.2

# 6. CARREGAMENTO E PREPARO DOS DADOS
# Substituir por carregamento real de df_modelo com colunas 'headline' e 'eventRet'
# df = pd.read_csv('dados.csv', sep='|')
# df_modelo = df.dropna(subset=['headline','eventRet'])
# df_modelo['sentiment'] = (df_modelo['eventRet'] >= 0).astype(int)

# Tokenização e padding
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(df_modelo['headline'])
sequences = tokenizer.texts_to_sequences(df_modelo['headline'])
X = pad_sequences(sequences, maxlen=SEQ_LENGTH, padding='post', truncating='post')
y = df_modelo['sentiment'].values

# Split treino+val / teste
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Divisão interna treino / validação
n_val = int(VAL_SPLIT * X_train_val.shape[0])
X_val = X_train_val[:n_val];   y_val = y_train_val[:n_val]
X_train = X_train_val[n_val:];  y_train = y_train_val[n_val:]

# 7. PIPELINE tf.data COM CACHE EM DISCO
train_ds = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
      .shuffle(buffer_size=10000)
      .cache("cache_train.tfdata")
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE)
)
val_ds = (
    tf.data.Dataset.from_tensor_slices((X_val, y_val))
      .cache("cache_val.tfdata")
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE)
)

# 8. DEFINIÇÃO DO MODELO (GRU)
def build_model() -> Sequential:
    model = Sequential()
    model.add(Embedding(
        input_dim=tokenizer.num_words,
        output_dim=EMBED_DIM,
        input_length=SEQ_LENGTH
    ))
    model.add(GRU(
        units=RNN_UNITS,
        dropout=0.2,
        recurrent_dropout=0.2
    ))
    model.add(Dense(1, activation='sigmoid', dtype='float32'))
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return model

model = build_model()

# 9. CALLBACKS DE TREINO
es = EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True
)
rlrp = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=1,
    verbose=1
)

# 10. TREINAMENTO
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[es, rlrp],
    verbose=1
)

# 11. PLOT DE MÉTRICAS
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'],     'o-', label='Train Loss')
plt.plot(history.history['val_loss'], 'o-', label='Val Loss')
plt.title('Curva de Loss')
plt.xlabel('Época'); plt.ylabel('Binary Crossentropy')
plt.legend(); plt.grid(True); plt.tight_layout()
plt.savefig('loss_curve.png', dpi=300); plt.show()

plt.figure(figsize=(8,4))
plt.plot(history.history['accuracy'],     'o-', label='Train Acc')
plt.plot(history.history['val_accuracy'], 'o-', label='Val Acc')
plt.title('Curva de Acurácia')
plt.xlabel('Época'); plt.ylabel('Acurácia')
plt.legend(); plt.grid(True); plt.tight_layout()
plt.savefig('accuracy_curve.png', dpi=300); plt.show()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

1. **Download manual do GloVe (100d):**

Acesse o link abaixo e baixe o arquivo necessário:

🔗 [https://nlp.stanford.edu/data/glove.6B.zip](https://nlp.stanford.edu/data/glove.6B.zip)

2. **Etapas:**

* Extraia o conteúdo do `.zip` baixado.
* Copie o arquivo `glove.6B.100d.txt` para o diretório onde seu script está localizado (ou especifique o caminho completo no `open()`).
* O arquivo ocupa cerca de 350 MB.


In [ ]:
# 1. VARIÁVEIS DE AMBIENTE
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

# 2. IMPORTS
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Embedding, GRU, Dense, Input, Bidirectional,
    SpatialDropout1D, Layer, GlobalAveragePooling1D
)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import mixed_precision
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 3. LIMPEZA DE SESSÃO
K.clear_session()

# 4. PRECISÃO MISTA
mixed_precision.set_global_policy('mixed_float16')

# 5. HYPERPARÂMETROS
VOCAB_SIZE   = 10000
SEQ_LENGTH   = 50
EMBED_DIM    = 100  # Para uso com GloVe 100d
RNN_UNITS    = 64
BATCH_SIZE   = 64
EPOCHS       = 10
VAL_SPLIT    = 0.2

# 6. CARREGAMENTO E PREPARO DOS DADOS
# df_modelo = pd.read_csv('dados.csv', sep='|').dropna(subset=['headline', 'eventRet'])
# df_modelo['sentiment'] = (df_modelo['eventRet'] >= 0).astype(int)

tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(df_modelo['headline'])
sequences = tokenizer.texts_to_sequences(df_modelo['headline'])
X = pad_sequences(sequences, maxlen=SEQ_LENGTH, padding='post', truncating='post')
y = df_modelo['sentiment'].values

X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=VAL_SPLIT, stratify=y_train_val, random_state=42
)

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# 7. MATRIZ DE EMBEDDING PRÉ-TREINADA (GloVe)
embedding_index = {}
with open("glove.6B.100d.txt", encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = vector

embedding_matrix = np.zeros((VOCAB_SIZE, EMBED_DIM))
for word, i in tokenizer.word_index.items():
    if i >= VOCAB_SIZE:
        continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# 8. CAMADA DE ATENÇÃO PERSONALIZADA
class AttentionLayer(Layer):
    def __init__(self):
        super().__init__()

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros", trainable=True)
        super().build(input_shape)

    def call(self, x):
        e = tf.nn.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
        a = tf.nn.softmax(e, axis=1)
        output = tf.reduce_sum(x * a, axis=1)
        return output

# 9. DEFINIÇÃO DO MODELO COM BIDIRECTIONAL, EMBEDDING FIXO, ATTENTION
def build_model() -> Model:
    inputs = Input(shape=(SEQ_LENGTH,))
    x = Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=EMBED_DIM,
        weights=[embedding_matrix],
        input_length=SEQ_LENGTH,
        trainable=False
    )(inputs)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(RNN_UNITS, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(x)
    x = AttentionLayer()(x)
    outputs = Dense(1, activation='sigmoid', dtype='float32')(x)
    model = Model(inputs, outputs)
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return model

model = build_model()

# 10. CALLBACKS
es = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, verbose=1)

# 11. TREINAMENTO
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[es, rlrp], verbose=1)

# 12. PLOT DE MÉTRICAS
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], 'o-', label='Train Loss')
plt.plot(history.history['val_loss'], 'o-', label='Val Loss')
plt.title('Curva de Loss')
plt.xlabel('Época'); plt.ylabel('Binary Crossentropy')
plt.legend(); plt.grid(True); plt.tight_layout()
plt.savefig('loss_curve.png', dpi=300); plt.show()

plt.figure(figsize=(8,4))
plt.plot(history.history['accuracy'], 'o-', label='Train Acc')
plt.plot(history.history['val_accuracy'], 'o-', label='Val Acc')
plt.title('Curva de Acurácia')
plt.xlabel('Época'); plt.ylabel('Acurácia')
plt.legend(); plt.grid(True); plt.tight_layout()
plt.savefig('accuracy_curve.png', dpi=300); plt.show()


In [ ]:
import os
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# 1. Configurações de hardware e sessão
os.environ["CUDA_VISIBLE_DEVICES"] = ""   # força CPU
K.clear_session()

# 2. Hiperparâmetros
vocabulary_size = 10000   # conforme tokenizador
input_length    = 50      # comprimento das sequências
embed_dim       = 64      # dimensão do embedding
lstm_units      = 32      # unidades LSTM
batch_size      = 16      # reduzido para economia de memória
epochs          = 10

# 3. Definição do modelo
def create_model():
    model = Sequential()
    model.add(Embedding(
        input_dim=vocabulary_size,
        output_dim=embed_dim,
        input_length=input_length
    ))
    model.add(LSTM(
        units=lstm_units,
        dropout=0.2,
        recurrent_dropout=0.2
    ))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return model

model_LSTM = create_model()

# 4. Treinamento com early stopping
es = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
history = model_LSTM.fit(
    X_train_LSTM,
    y_train_LSTM,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.2,
    callbacks=[es],
    verbose=1
)

# 5. Plot das curvas de treinamento

# 5.1 Curva de Loss
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'],     label='Train Loss', marker='o')
plt.plot(history.history['val_loss'], label='Val Loss',   marker='o')
plt.title('Curva de Loss durante o Treinamento')
plt.xlabel('Época')
plt.ylabel('Binary Crossentropy')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('loss_curve.png', dpi=300)
plt.show()

# 5.2 Curva de Acurácia
plt.figure(figsize=(8,4))
plt.plot(history.history['accuracy'],      label='Train Acc', marker='o')
plt.plot(history.history['val_accuracy'],  label='Val Acc',   marker='o')
plt.title('Curva de Acurácia durante o Treinamento')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('accuracy_curve.png', dpi=300)
plt.show()



In [ ]:
# Parte 6. Construção e treinamento do modelo LSTM (ajustado para menor consumo de memória)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
import os

# Forçar CPU para evitar OOM na GPU (caso deseje GPU, remova estas linhas)
os.environ["CUDA_VISIBLE_DEVICES"] = ""
K.clear_session()

def create_model(input_length: int = 50) -> Sequential:
    model = Sequential()
    # Embedding com dimensão reduzida
    model.add(Embedding(
        input_dim=vocabulary_size,   # conforme definido na etapa anterior
        output_dim=64,               # reduzido de 128 para 64
        input_length=input_length
    ))
    # LSTM com menos unidades
    model.add(LSTM(
        units=64,                    # reduzido de 64 para 32
        dropout=0.2,
        recurrent_dropout=0.2
    ))
    # Camada de saída
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return model

model_LSTM = create_model(input_length=50)

# Treinamento com batch menor
es = EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True
)
history = model_LSTM.fit(
    X_train_LSTM,
    y_train_LSTM,
    epochs=10,
    batch_size=64,
    validation_split=0.2,
    callbacks=[es],
    verbose=1
)

# 5. Plot das curvas de treinamento

# 5.1 Curva de Loss
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'],     label='Train Loss', marker='o')
plt.plot(history.history['val_loss'], label='Val Loss',   marker='o')
plt.title('Curva de Loss durante o Treinamento')
plt.xlabel('Época')
plt.ylabel('Binary Crossentropy')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('loss_curve.png', dpi=300)
plt.show()

# 5.2 Curva de Acurácia
plt.figure(figsize=(8,4))
plt.plot(history.history['accuracy'],      label='Train Acc', marker='o')
plt.plot(history.history['val_accuracy'],  label='Val Acc',   marker='o')
plt.title('Curva de Acurácia durante o Treinamento')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('accuracy_curve.png', dpi=300)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    roc_auc_score,
    roc_curve,
    precision_recall_curve,
    auc as sk_auc
)
from sklearn.metrics import ConfusionMatrixDisplay

# --- Etapa 7: Avaliação do modelo LSTM ---

# 1. Geração das predições binárias
y_train_prob = model_LSTM.predict(X_train_LSTM).flatten()
y_test_prob  = model_LSTM.predict(X_test_LSTM).flatten()

y_train_pred = (y_train_prob > 0.5).astype(int)
y_test_pred  = (y_test_prob  > 0.5).astype(int)

# 2. Cálculo e exibição de acurácia
print("\n=== Avaliação LSTM ===")
print("Acurácia Treino:", accuracy_score(y_train_LSTM, y_train_pred))
print("Acurácia Teste:",  accuracy_score(y_test_LSTM, y_test_pred))

# 3. Relatório de classificação
print("\nClassification Report (Teste):")
print(classification_report(y_test_LSTM, y_test_pred, digits=4))

# 4. Matriz de confusão (counts)
cm = confusion_matrix(y_test_LSTM, y_test_pred)
disp = ConfusionMatrixDisplay(cm, display_labels=["Negativo (0)", "Positivo (1)"])
fig, ax = plt.subplots(figsize=(5,5))
disp.plot(cmap='Blues', ax=ax, colorbar=False)
ax.set_title("Matriz de Confusão — Teste")
plt.show()

# 5. Matriz de confusão normalizada
cm_norm = cm.astype(float) / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(5,5))
sns.heatmap(cm_norm, annot=True, fmt=".2f", cmap="Blues",
            xticklabels=["Pred 0","Pred 1"], yticklabels=["True 0","True 1"])
plt.title("Matriz de Confusão Normalizada")
plt.xlabel("Classe Predita")
plt.ylabel("Classe Verdadeira")
plt.tight_layout()
plt.show()

# 6. Cálculo de ROC AUC e Curva ROC
roc_auc = roc_auc_score(y_test_LSTM, y_test_prob)
fpr, tpr, _ = roc_curve(y_test_LSTM, y_test_prob)
plt.figure(figsize=(6,4))
plt.plot(fpr, tpr, label=f"ROC AUC = {roc_auc:.4f}")
plt.plot([0,1], [0,1], 'k--', linewidth=0.8)
plt.title("Curva ROC — Teste")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()

# 7. Curva Precision–Recall
precision, recall, _ = precision_recall_curve(y_test_LSTM, y_test_prob)
pr_auc = sk_auc(recall, precision)
plt.figure(figsize=(6,4))
plt.plot(recall, precision, label=f"PR AUC = {pr_auc:.4f}")
plt.title("Curva Precision–Recall — Teste")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(loc="upper right")
plt.grid(True)
plt.tight_layout()
plt.show()



In [ ]:
#Parte 7. Treinamento com early stopping
es = EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True
)
history = model_LSTM.fit(
    X_train_LSTM, y_train_LSTM,
    epochs=5,
    batch_size=64,
    validation_split=0.2,
    callbacks=[es],
    verbose=1
)
#Parte 8. Avaliação do modelo

from sklearn.metrics import accuracy_score, confusion_matrix

# Predição em treino e teste
y_train_pred = (model_LSTM.predict(X_train_LSTM) > 0.5).astype(int).flatten()
y_test_pred  = (model_LSTM.predict(X_test_LSTM)  > 0.5).astype(int).flatten()

# Exibe métricas
print("\n=== Avaliação LSTM ===")
print("Acurácia Treino:", accuracy_score(y_train_LSTM, y_train_pred))
print("Acurácia Teste:",  accuracy_score(y_test_LSTM, y_test_pred))
print("Matriz de Confusão (Teste):\n", confusion_matrix(y_test_LSTM, y_test_pred))

#Parte 9. Aplicação ao DataFrame completo e correlação
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Verifica coluna 'headline'
assert 'headline' in data_df.columns, "Coluna 'headline' não encontrada em data_df."

# Preprocessa textos e faz predição
seq_data   = tokenizer.texts_to_sequences(data_df["headline"].astype(str))
X_seq_data = pad_sequences(seq_data, maxlen=50, padding='post', truncating='post')
y_sentiment_pred = (model_LSTM.predict(X_seq_data) > 0.5).astype(int).flatten()

# Inclui no DataFrame e calcula correlação
data_df["sentiment_LSTM"] = y_sentiment_pred
correlacao = data_df["eventRet"].corr(data_df["sentiment_LSTM"])
print(f"Correlação entre sentimento LSTM e retorno: {correlacao:.4f}")

# Salvamento em CSV
data_df.to_csv("/content/Step4_Resultado_LSTM.csv", sep="|", index=False)
print("✅ Resultados salvos com sucesso.")



In [ ]:
#  Etapa 5 – Aplicação do modelo LSTM sobre data_df
# Verifica se data_df possui coluna 'headline'
assert 'headline' in data_df.columns, "Coluna 'headline' não encontrada em data_df."

# Preprocessamento
seq_data = tokenizer.texts_to_sequences(data_df["headline"].astype(str))
X_seq_data = pad_sequences(seq_data, maxlen=50)

# Predição
y_sentiment_pred = (model_LSTM.predict(X_seq_data) > 0.5).astype(int).flatten()

# Adiciona ao dataframe
data_df["sentiment_LSTM"] = y_sentiment_pred



In [ ]:
# Etapa 6 – Correlação com retorno
correlacao = data_df["eventRet"].corr(data_df["sentiment_LSTM"])
print(f"Correlação entre sentimento LSTM e retorno: {correlacao:.4f}")


In [ ]:
# Etapa 7 – Exportação final (opcional)
data_df.to_csv("/content/Step4_Resultado_LSTM.csv", sep="|", index=False)
print("✅ Resultados salvos com sucesso.")

<a id='4.3'></a>
## 4.3 - Modelo não supervisionado baseado em léxico (VADER)

Modelo não supervisionado: Foi implementado um modelo de análise de sentimento baseado na técnica léxica VADER (Valence Aware Dictionary for Sentiment Reasoning), originalmente em inglês, adaptado para o contexto financeiro em português. Para isso, utilizou-se um dicionário de polaridades construído com apoio de modelos de linguagem generativos (ChatGPT-4 Mini e Gemini 2.5), contendo termos financeiros com valência atribuída manualmente e armazenada em formato JSON.
Os valores de polaridade foram normalizados na escala de -4 a +4 e integrados ao léxico original do VADER. Com isso, o modelo passou a reconhecer e ponderar termos específicos do domínio financeiro, aumentando sua sensibilidade semântica.

O VADER (Valence Aware Dictionary for Sentiment Reasoning) é um modelo pré-construído de análise de sentimentos incluído no pacote NLTK.

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# Inicializa o analisador de sentimento VADER
sia = SentimentIntensityAnalyzer()

# Aplica o modelo padrão VADER na coluna de manchetes
sentiments_data["sentiment_vader"] = sentiments_data["headline"].apply(
    lambda x: sia.polarity_scores(str(x))["compound"]
)

# Visualização
print(sentiments_data[["headline", "sentiment_vader"]].head())


In [ ]:
import json
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Baixar o VADER, caso ainda não tenha feito
nltk.download('vader_lexicon')

# Carregar o VADER padrão
sia = SentimentIntensityAnalyzer()

# Carregar termos do arquivo JSON
with open('/content/TERMOS.json', encoding='utf-8') as f:
    termos = json.load(f)

# Converter para DataFrame para facilitar manipulação
df_lexico = pd.DataFrame(termos)

# Remover duplicatas baseadas no termo
df_lexico = df_lexico.drop_duplicates(subset="termo")

# Construir o dicionário: termo → polaridade
lexico_customizado = dict(zip(df_lexico['termo'], df_lexico['polaridade']))

# Escalar os valores para mesma faixa do VADER (~ -4 a +4)
max_pos = max([v for v in lexico_customizado.values() if v > 0])
min_neg = min([v for v in lexico_customizado.values() if v < 0])

lexico_ajustado = {}
for termo, valor in lexico_customizado.items():
    if valor > 0:
        lexico_ajustado[termo] = valor / max_pos * 4
    else:
        lexico_ajustado[termo] = valor / abs(min_neg) * -4


In [ ]:
# Atualizar o léxico original do VADER
sia.lexicon.update(lexico_ajustado)


In [ ]:
# Aplicando o VADER nas manchetes do dataset de notícias
sentiments_data['sentiment_lex'] = sentiments_data['headline'].apply(lambda x: sia.polarity_scores(x)['compound'])

# Visualização dos resultados
sentiments_data[['headline', 'sentiment_lex']].head()



In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.hist(sentiments_data['sentiment_lex'], bins=30, color='skyblue', edgecolor='black')
plt.title('Distribuição dos Sentimentos (Léxico Financeiro VADER)')
plt.xlabel('Pontuação de Sentimento')
plt.ylabel('Frequência')
plt.grid(True)
plt.show()


In [ ]:
corrlation = data_df['eventRet'].corr(data_df['sentiment_lex'])
print(corrlation)

In [ ]:
plt.scatter(data_df['sentiment_lex'],data_df['eventRet'], alpha=0.5)
plt.title('Scatter Between Event return and sentiments-all data')
plt.ylabel('Event Return')
plt.xlabel('Sentiments')
plt.show()

In [ ]:
# Visualização dos dois principais tickers
for ticker in tickers_top[:2]:
    subset = data_df[data_df['ticker'] == ticker].dropna()

    fig, axs = plt.subplots(1, 2, figsize=(14, 5))

    axs[0].scatter(subset['sentiment_lex'], subset['eventRet'], alpha=0.5)
    axs[0].set_title(f'{ticker} - Lexico x Retorno')
    axs[0].set_xlabel("Sentimento (Léxico)")
    axs[0].set_ylabel("Retorno")

    axs[1].scatter(subset['sentiment_LSTM'], subset['eventRet'], alpha=0.5)
    axs[1].set_title(f'{ticker} - LSTM x Retorno')
    axs[1].set_xlabel("Sentimento (LSTM)")
    axs[1].set_ylabel("Retorno")

    plt.tight_layout()
    plt.show()


<a id='4.4'></a>
## 4.4 Análise Exploratória e Comparação dos Modelos de Sentimento

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Carregando DataFrame consolidado
data_df = pd.read_csv("/content/Step4_Resultado_LSTM.csv", sep="|")

# Seleciona as colunas relevantes
colunas_modelos = [
    'sentiment_lex',
    'sentiment_LSTM',
    'sentiment_FinBERT_PT_BR',
    'sentiment_FinBERT_Turing',
    'sentiment_Multilingual_Sentiment',
    'eventRet'
]

# Remove valores ausentes
df_corr = data_df[colunas_modelos].dropna()

# Matriz de correlação
correlacoes = df_corr.corr()

# Visualização com heatmap
plt.figure(figsize=(6, 6))
sns.heatmap(correlacoes[['eventRet']], annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Correlação entre Sentimentos e Retorno de Evento")
plt.show()


In [ ]:
# Análise de correlação por ticker
corr_data = []

for ticker in data_df['ticker'].dropna().unique():
    subset = data_df[data_df['ticker'] == ticker].dropna(subset=colunas_modelos)
    if subset.shape[0] >= 40:
        linha = [ticker]
        for col in colunas_modelos[:-1]:  # Exceto 'eventRet'
            linha.append(subset['eventRet'].corr(subset[col]))
        corr_data.append(linha)

# Criação do DataFrame
colunas_corr = ['ticker'] + [f'corr_{col}' for col in colunas_modelos[:-1]]
corr_df = pd.DataFrame(corr_data, columns=colunas_corr).set_index('ticker')
corr_df.head()


In [ ]:
# Exibe os 5 ativos com maior retorno absoluto para comparação
tickers_top = corr_df['corr_sentiment_LSTM'].abs().sort_values(ascending=False).head(5).index.tolist()

corr_df.loc[tickers_top].plot(kind='bar', figsize=(12, 6))
plt.title("Correlação por Ticker - Modelos de Sentimento")
plt.ylabel("Correlação com Retorno de Evento")
plt.xlabel("Ticker")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


<a id='5'></a>
# 5. Estratégia de Negociação Baseada em Sentimento

In [ ]:
!pip install backtrader yfinance --quiet
import yfinance as yf
import pandas as pd
import backtrader as bt
import matplotlib.pyplot as plt


Os dados de sentimento podem ser usados ​​de diferentes maneiras para a estratégia de negociação. As pontuações de sentimento podem ser usadas como um sinal direcional e, idealmente, criar uma carteira long-short, comprando as ações com pontuação positiva e vendendo as ações com pontuação negativa. Os sentimentos também podem ser usados ​​como recursos adicionais, além de outros recursos (como ações correlacionadas e indicadores técnicos), em um modelo de aprendizado supervisionado para prever o preço ou elaborar uma estratégia de negociação.

Na estratégia de negociação neste estudo de caso, compramos e vendemos ações de acordo com os sentimentos atuais das ações:

* Compre uma ação quando a mudança na pontuação de sentimento (pontuação de sentimento atual - pontuação de sentimento anterior) for maior que 0,5 e venda uma ação quando a mudança na pontuação de sentimento for menor que -0,5.
* Além disso, verificamos a média móvel de 15 dias durante a compra e venda e compramos ou vendemos em uma unidade de 100.
Obviamente, pode haver muitas maneiras de criar uma estratégia de negociação baseada em sentimentos, variando o limite ou alterando o número de unidades com base no dinheiro inicial disponível.

Usamos sentimentos baseados em léxico para a estratégia de negociação

<a id='5.1'></a>
## 5.1.  Definição da Estratégia com Backtrader
Aqui, usamos o Backtrader, uma API baseada em Python para escrever e testar estratégias de negociação. O Backtrader permite que você se concentre em escrever estratégias de negociação, indicadores e analisadores reutilizáveis, em vez de ter que gastar tempo construindo infraestrutura. Temos uma estrutura conveniente para testar e escrever nossa estratégia de negociação. Usamos o código Quickstart na documentação (consulte https://www.backtrader.com/docu/quickstart/quickstart/ ) como base e o modificamos para incluir as pontuações de sentimento.

Implementamos uma estratégia simples para comprar se a pontuação de sentimento do dia anterior aumentar em 0,5 em relação ao último dia e vender se diminuir em 0,5.

A função a seguir contém duas classes:

Sentimento:
SentimentStrat: A função "next" desta classe implementa a estratégia de negociação real.

## 1. Coleta de Dados de Texto e Mercado
A primeira etapa consistiu na coleta automatizada de manchetes de notícias relacionadas a quatro ativos do mercado acionário brasileiro: VALE3.SA, PETR4.SA, ITUB4.SA e BBAS3.SA. A extração foi realizada por meio de requisições ao serviço RSS do Google News, utilizando expressões regulares parametrizadas com múltiplas formas de referência a cada ticker. O período considerado foi de 1º de janeiro de 2025 a 4 de maio de 2025, com segmentação mensal para respeitar limites de requisição e evitar sobrecarga nos servidores.

In [ ]:
# === IMPORTAÇÃO DE BIBLIOTECAS ===
import os
import json
import zipfile
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta

# === CONFIGURAÇÃO INICIAL ===
tickers_termos = {
    'VALE3.SA': ['VALE3', 'VALE3.SA', 'BVMF:VALE3'],
    'PETR4.SA': ['PETR4', 'PETR4.SA', 'BVMF:PETR4'],
    'ITUB4.SA': ['ITUB4', 'ITUB4.SA', 'BVMF:ITUB4'],
    'BBAS3.SA': ['BBAS3', 'BBAS3.SA', 'BVMF:BBAS3']
}

# Período de coleta
inicio = datetime.strptime('2025-01-01', '%Y-%m-%d')
fim    = datetime.strptime('2025-05-04', '%Y-%m-%d')

# Diretório de saída
data_suffix = datetime.now().strftime("%Y%m%d_%H%M%S")
saida_dir = f"noticias_json_{data_suffix}"
os.makedirs(saida_dir, exist_ok=True)

# === GERA INTERVALOS MENSAIS ENTRE DUAS DATAS ===
def gerar_periodos(start, end):
    periodos = []
    atual = start
    while atual < end:
        fim_mes = (atual + relativedelta(months=1)) - relativedelta(days=1)
        if fim_mes > end:
            fim_mes = end
        periodos.append((atual, fim_mes))
        atual = fim_mes + relativedelta(days=1)
    return periodos

# === COLETA DE NOTÍCIAS POR TICKER E PERÍODO ===
def coletar_noticias(ticker, termos, dt_ini, dt_fim):
    query = "(" + " OR ".join([f'"{t}"' if ' ' in t else t for t in termos]) + ")"
    url = (
        'https://news.google.com/rss/search'
        f'?q={query}+after:{dt_ini.strftime("%Y-%m-%d")}+before:{dt_fim.strftime("%Y-%m-%d")}'
        '&hl=pt-BR&gl=BR&ceid=BR:pt-419'
    )

    noticias = []
    try:
        resp = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=10)
        soup = BeautifulSoup(resp.content, 'xml')

        for item in soup.find_all('item'):
            pub = datetime.strptime(item.pubDate.text, '%a, %d %b %Y %H:%M:%S %Z')
            if not (dt_ini <= pub <= dt_fim):
                continue
            titulo = item.title.text
            link = item.link.text
            resumo = BeautifulSoup(item.description.text, 'html.parser').get_text()
            if any(term.lower() in (titulo + link).lower() for term in termos):
                noticias.append({
                    'ticker': ticker,
                    'date': pub.strftime('%Y-%m-%d'),
                    'time': pub.strftime('%H:%M'),
                    'headline': titulo,
                    'summary': resumo,
                    'source': 'Google News RSS',
                    'url': link,
                    'language': 'pt',
                    'scraped_at': datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
                })
    except Exception as e:
        print(f"[ERRO] {ticker} ({dt_ini.date()} a {dt_fim.date()}): {e}")
    return noticias

# === EXECUÇÃO DA COLETA ===
periodos = gerar_periodos(inicio, fim)

for ticker, termos in tickers_termos.items():
    todas = []
    for dt_ini, dt_fim in periodos:
        todas += coletar_noticias(ticker, termos, dt_ini, dt_fim)

    if todas:
        path_json = os.path.join(saida_dir, f"{ticker}_noticias_{data_suffix}.json")
        with open(path_json, "w", encoding="utf-8") as f:
            json.dump(todas, f, indent=2, ensure_ascii=False)
        print(f"[✔️] {ticker}: {len(todas)} notícias salvas.")

# === COMPACTAÇÃO FINAL ===
zip_path = f"noticias_tickers_{data_suffix}.zip"
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
    for arq in os.listdir(saida_dir):
        zipf.write(os.path.join(saida_dir, arq), arcname=arq)

print(f"\n Arquivo ZIP final criado: {zip_path}")


Como resultado, foram coletadas:
* 367 manchetes para VALE3.SA
* 384 manchetes para PETR4.SA
* 245 manchetes para ITUB4.SA
* 301 manchetes para BBAS3.SA

Cada registro inclui data e hora da publicação, manchete, resumo da notícia, fonte, URL e idioma detectado. As notícias foram armazenadas em arquivos JSON e posteriormente estruturadas em um DataFrame padronizado para análise.


In [ ]:
from transformers import pipeline

def calcular_polaridade_continua(model_id, n_classes, textos):
    """
    Aplica modelo Hugging Face e transforma saída de classificação em valor contínuo entre -1 e +1.
    """
    # Define pipeline de sentimento com saída de probabilidade por classe
    classifier = pipeline("text-classification", model=model_id, return_all_scores=True, truncation=True)

    resultados = []

    for texto in tqdm(textos, desc=f"Analisando com {model_id}"):
        try:
            # Obtém as probabilidades para cada classe
            scores = classifier(str(texto))[0]

            # Mapeia classes padrão: assume 3 classes [NEG, NEU, POS]
            if n_classes == 3:
                probs = {s['label'].lower(): s['score'] for s in scores}

                # Normaliza nomes típicos de labels para consistência
                neg = next((v for k, v in probs.items() if 'neg' in k), 0)
                neu = next((v for k, v in probs.items() if 'neu' in k), 0)
                pos = next((v for k, v in probs.items() if 'pos' in k), 0)

                # Calcula escore contínuo: -1*neg + 0*neu + 1*pos = pos - neg
                score = pos - neg
            else:
                raise ValueError("Modelo com número de classes diferente de 3 não suportado.")

        except Exception as e:
            print(f"[ERRO interno] Texto: {texto[:50]}... -> {e}")
            score = 0.0  # fallback neutro

        resultados.append(score)

    return resultados


In [ ]:
import os
import json
import pandas as pd
from zipfile import ZipFile

# Caminho para o arquivo ZIP gerado anteriormente
zip_path = "/content/noticias_tickers_20250504_203905.zip"
extract_path = "/content/noticias_2025"

# Extrai o conteúdo do ZIP
os.makedirs(extract_path, exist_ok=True)
with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Consolida todos os arquivos JSON em um único DataFrame
noticias_lista = []
for arquivo in os.listdir(extract_path):
    if arquivo.endswith(".json"):
        caminho = os.path.join(extract_path, arquivo)
        with open(caminho, encoding="utf-8") as f:
            dados = json.load(f)
            noticias_lista.extend(dados)

# Cria DataFrame único com as notícias
df_noticias = pd.DataFrame(noticias_lista)
df_noticias['date'] = pd.to_datetime(df_noticias['date']).dt.date  # normaliza data
df_noticias = df_noticias[['date', 'ticker', 'headline']].dropna()

# Visualização
print(f"[✔️] Total de registros carregados: {len(df_noticias)}")
df_noticias.head()


Carregamento e Consolidação dos Arquivos de Notícias

Aplicar os Modelos de Sentimento (Já Treinados)

## 2. Classificação de Sentimento com Modelos NLP


As manchetes foram submetidas aos cinco modelos de análise de sentimento descritos anteriormente:

LSTM supervisionado (baseado em retorno de evento)

VADER com léxico financeiro em português

FinBERT_PT_BR

FinBERT_Turing

Multilingual_Sentiment

Todos os modelos foram aplicados com saída contínua no intervalo [–1, +1], por meio de transformação dos logits com softmax (nos modelos Hugging Face) ou diretamente por cálculo de escore composto (no caso do VADER).

In [ ]:
# === GARANTE QUE AS COLUNAS ORIGINAIS NÃO SERÃO PERDIDAS ===
colunas_originais = df_noticias.columns.tolist()

# === MODELO LSTM ===
print(" Aplicando modelo LSTM...")
seq = tokenizer.texts_to_sequences(df_noticias['headline'].astype(str))
X_seq = pad_sequences(seq, maxlen=50)
df_noticias['sentiment_LSTM_raw'] = model_LSTM.predict(X_seq, verbose=0).flatten()
df_noticias['sentiment_LSTM'] = (2 * df_noticias['sentiment_LSTM_raw']) - 1  # Normaliza para [-1, 1]

# === MODELO VADER (LÉXICO FINANCEIRO) ===
print(" Aplicando modelo VADER com léxico financeiro...")
df_noticias['sentiment_lex'] = df_noticias['headline'].apply(
    lambda x: sia.polarity_scores(str(x))['compound']
)

# === MODELOS HUGGING FACE COM TRATAMENTO DE INVERSÃO ===
modelos_hf = {
    "FinBERT_PT_BR": {
        "repo": "lucas-leme/FinBERT-PT-BR",
        "n_classes": 3,
        "inverter_escala": True  # ← Inverter resultado
    },
    "FinBERT_Turing": {
        "repo": "turing-usp/FinBertPTBR",
        "n_classes": 3,
        "inverter_escala": True  # ← Inverter resultado
    },
    "Multilingual_Sentiment": {
        "repo": "tabularisai/multilingual-sentiment-analysis",
        "n_classes": 3,
        "inverter_escala": False
    }
}

for nome_modelo, config in modelos_hf.items():
    print(f" Aplicando {nome_modelo}...")
    try:
        coluna_saida = f"sentiment_{nome_modelo}"
        valores = calcular_polaridade_continua(
            model_id=config["repo"],
            n_classes=config["n_classes"],
            textos=df_noticias["headline"]
        )

        # Inversão se necessário
        if config.get("inverter_escala", False):
            valores = [-1 * v for v in valores]

        df_noticias[coluna_saida] = valores

    except Exception as e:
        print(f"[ERRO] {nome_modelo}: {e}")

# === VERIFICAÇÃO FINAL ===
print("Colunas após aplicação de todos os modelos:")
print(df_noticias.columns.tolist())


Agregação Diária por Ticker

In [ ]:

# ✅ Validação: imprime colunas disponíveis
print("Colunas disponíveis em df_noticias:", df_noticias.columns.tolist())


In [ ]:
# Agrupamento por data e ticker, ordenando por ordem cronológica para manter consistência
agrupado = df_noticias.sort_values(['date', 'ticker']).groupby(['date', 'ticker'])

# Agregação dos sentimentos por média, e preservação da primeira notícia e resumo
df_diario = agrupado.agg({
    'headline': 'first',
    'sentiment_lex': 'mean',
    'sentiment_LSTM': 'mean',
    'sentiment_FinBERT_PT_BR': 'mean',
    'sentiment_FinBERT_Turing': 'mean',
    'sentiment_Multilingual_Sentiment': 'mean',
}).reset_index()

# Visualização e confirmação
print(f"[✔️] df_diario gerado com {df_diario.shape[0]} linhas e {df_diario.shape[1]} colunas.")
display(df_diario.head())

In [ ]:
import pandas as pd
from IPython.display import display, Markdown
import seaborn as sns

# 1. Verifica estrutura do DataFrame
colunas = df_diario.columns.tolist()
tickers = df_diario['ticker'].unique()
data_inicio = df_diario['date'].min()
data_fim = df_diario['date'].max()

# 2. Calcula estatísticas (mínimo, máximo e média) por modelo
estatisticas = df_diario[[
    'sentiment_lex',
    'sentiment_LSTM',
    'sentiment_FinBERT_PT_BR',
    'sentiment_FinBERT_Turing',
    'sentiment_Multilingual_Sentiment'
]].describe().loc[['min', 'max', 'mean']].round(3)

# 3. Mostra resumo textual
display(Markdown(f"""
### 📄 Estrutura do DataFrame

**Colunas presentes:**
`{', '.join(colunas)}`

**Tickers únicos encontrados:**
{', '.join(tickers)}

**Intervalo de datas disponíveis:**
De **{data_inicio}** até **{data_fim}**
"""))

# 4. Mostra estatísticas com formatação
cmap = sns.light_palette("green", as_cmap=True)

display(
    estatisticas.style
        .background_gradient(cmap=cmap)
        .format(precision=3)
        .set_caption("📊 Estatísticas dos Sentimentos (mín, máx, média)")
        .set_properties(**{'font-size': '10pt', 'width': '80px'})
)


In [ ]:
from IPython.display import display, HTML

# Lista de tickers
tickers = ['BBAS3.SA', 'ITUB4.SA', 'PETR4.SA', 'VALE3.SA']

# Colunas de sentimento
colunas_sentimentos = [
    'sentiment_lex',
    'sentiment_LSTM',
    'sentiment_FinBERT_PT_BR',
    'sentiment_FinBERT_Turing',
    'sentiment_Multilingual_Sentiment'
]

# Armazena as tabelas formatadas
html_tabelas = []

for ticker in tickers:
    df_ticker = df_diario[df_diario['ticker'] == ticker]

    if len(df_ticker) < 2:
        continue

    matriz_corr = df_ticker[colunas_sentimentos].corr(method='pearson')

    styler = matriz_corr.style\
        .background_gradient(cmap='coolwarm')\
        .format(precision=2)\
        .set_caption(f"<b>{ticker}</b>")\
        .set_properties(**{
            'font-size': '9pt',
            'width': '60px'
        })

    html_tabelas.append(styler.to_html())

# Cria layout horizontal com CSS flexbox
html_final = f"""
<div style="display: flex; gap: 20px;">
    {''.join(f'<div>{t}</div>' for t in html_tabelas)}
</div>
"""

# Exibe todas as matrizes lado a lado
display(HTML(html_final))


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set_style("whitegrid")  # fundo com grid discreto
plt.rcParams.update({
    'axes.titlesize': 14,
    'axes.titleweight': 'bold',
    'axes.labelsize': 12,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'lines.linewidth': 2,
    'grid.alpha': 0.6,
    'grid.linestyle': '--',
    'figure.figsize': (14, 6)
})

# Ativo em foco
ticker_exemplo = 'VALE3.SA'
df_exemplo = df_diario[df_diario['ticker'] == ticker_exemplo].copy()

# Média móvel para suavização dos dados
window = 5
for col in [
    'sentiment_lex', 'sentiment_LSTM', 'sentiment_FinBERT_PT_BR',
    'sentiment_FinBERT_Turing', 'sentiment_Multilingual_Sentiment'
]:
    df_exemplo[col + '_smoothed'] = df_exemplo[col].rolling(window=window, center=True).mean()

# Cores elegantes e consistentes
cores = {
    'sentiment_lex_smoothed': '#1f77b4',
    'sentiment_LSTM_smoothed': '#ff7f0e',
    'sentiment_FinBERT_PT_BR_smoothed': '#2ca02c',
    'sentiment_FinBERT_Turing_smoothed': '#d62728',
    'sentiment_Multilingual_Sentiment_smoothed': '#9467bd'
}

# Criação da figura
fig, ax = plt.subplots()

# Plotagem das curvas suavizadas
for col, cor in cores.items():
    ax.plot(df_exemplo['date'], df_exemplo[col], label=col.replace('_smoothed', ''), color=cor)

# Ajustes finais no layout
ax.set_title(f"Evolução dos Sentimentos – {ticker_exemplo}")
ax.set_xlabel("Data")
ax.set_ylabel("Sentimento Médio Diário")
ax.legend(frameon=False)
plt.tight_layout()
plt.show()



## Etapa 2: Execução da Estratégia Individual por Modelo de Sentimento

### Tentei incluir a estratégia de média móvel, mas não consegui.

Será realizada a simulação da estratégia para **cada coluna de sentimento** (`sentiment_lex`, `sentiment_LSTM`, etc.) **individualmente**. Como o intervalo de tempo da análise é curto (`2025-01-01` a `2025-05-04` ≈ 4 meses), utilizar **apenas a média móvel de 15 dias (SMA15)** pode limitar a responsividade da estratégia. Adotamos então variação da estratégia com múltiplas médias móveis:

| Nome      | Período | Finalidade                              |
| --------- | ------- | --------------------------------------- |
| **SMA5**  | 5 dias  | Curto prazo, mais sensível a reversões  |
| **SMA10** | 10 dias | Curto/médio prazo, menos ruído que SMA5 |
| **SMA15** | 15 dias | Referência padrão de tendência          |
| **SMA20** | 20 dias | Média mais "suave", evita sinais falsos |

---

### Estratégia por Média Móvel

Para cada uma dessas médias móveis, aplicaremos:

#### **Critério de Compra**

* > Compra: Δsentimento > 0.5 e Preço atual > média móvel (SMAx)
* > Venda: Δsentimento < -0.5 e Preço atual < média móvel (SMAx)


### Execuções planejadas

Para **cada modelo de sentimento** (`sentiment_lex`, `sentiment_LSTM`, etc.), a estratégia será executada **para cada média móvel separadamente**, totalizando:

```
5 modelos × 4 médias móveis = 20 simulações
```

Cada simulação terá:

* Valor inicial: R\$ 100.000
* Lote fixo: 100 ações
* Período: 2025-01-01 a 2025-05-04


#Coleta dos preços históricos com médias móveis

Código para coletar os preços históricos de fechamento via `yfinance`, calcular as médias móveis e deixar tudo pronto para cruzar com o DataFrame de sentimentos por ticker e data:

In [ ]:
# IMPORTAÇÃO DE BIBLIOTECAS

import yfinance as yf
import pandas as pd
import os

from google.colab import data_table
data_table.enable_dataframe_formatter()


# PARÂMETROS DE CONFIGURAÇÃO

# Tickers brasileiros analisados
tickers = ['VALE3.SA', 'PETR4.SA', 'ITUB4.SA', 'BBAS3.SA']

# Intervalo de datas da análise
start = '2025-01-01'
end   = '2025-05-04'

# Diretório de saída
os.makedirs("data", exist_ok=True)

# MAPA DOS DIAS DA SEMANA EM PORTUGUÊS

dias_semana_pt = {
    'Monday': 'Segunda-feira',
    'Tuesday': 'Terça-feira',
    'Wednesday': 'Quarta-feira',
    'Thursday': 'Quinta-feira',
    'Friday': 'Sexta-feira',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

# COLETA DOS DADOS HISTÓRICOS COM TRATAMENTO

dados_completos = []

for ticker in tickers:
    print(f" Coletando dados de: {ticker}")
    try:
        ticker_yf = yf.Ticker(ticker)
        dados = ticker_yf.history(start=start, end=end)

        # Validação: ignorar tickers com dados vazios
        if dados.empty:
            print(f" Dados indisponíveis para {ticker}, ignorado.")
            continue

        # Resetar índice (Date como coluna)
        dados = dados.reset_index()

        # Garantir consistência de datas e tipos
        dados['Date'] = pd.to_datetime(dados['Date'], errors='coerce')
        dados = dados.dropna(subset=['Date'])

        # Formatação de colunas auxiliares
        dados['Data'] = dados['Date'].dt.strftime('%Y-%m-%d')
        dados['Hora'] = dados['Date'].dt.strftime('%H:%M:%S')
        dados['Dia_da_semana'] = dados['Date'].dt.day_name().map(dias_semana_pt)
        dados['Ticker'] = ticker

        # Seleção e ordenação de colunas
        dados_formatado = dados[[
            'Data', 'Dia_da_semana', 'Open', 'High', 'Low', 'Close',
            'Volume', 'Dividends', 'Stock Splits', 'Ticker'
        ]]

        dados_completos.append(dados_formatado)
        print(f" {ticker}: {dados_formatado.shape[0]} registros coletados.")

    except Exception as e:
        print(f" Erro ao coletar dados de {ticker}: {e}")


# CONSOLIDAÇÃO E SALVAMENTO DOS RESULTADOS

if dados_completos:
    df_final = pd.concat(dados_completos, ignore_index=True)
    df_final = df_final.sort_values(by=['Ticker', 'Data'])

    # Caminhos de saída
    caminho_csv   = "data/ReturnData_2025.csv"
    caminho_excel = "data/ReturnData_2025.xlsx"

    # Salvamento em formatos compatíveis
    df_final.to_csv(caminho_csv, sep='|', index=False)
    df_final.to_excel(caminho_excel, index=False, engine='openpyxl')

    print(f"\n Arquivos salvos com sucesso:")
    print(f"  - CSV: {caminho_csv}")
    print(f"  - Excel: {caminho_excel}")

    # Exibição interativa no Colab
    display(data_table.DataTable(df_final))
else:
    print(" Nenhum dado foi coletado com sucesso.")

# Cálculo das Médias Móveis

Abaixo está o código para adicionar colunas de médias móveis ao DataFrame `df_final`, para cada `Ticker` individualmente, usando os valores da coluna `Close`.

Médias móveis incluídas:

* SMA5: 5 dias (curto prazo)
* SMA10: 10 dias
* SMA15: 15 dias (padrão da estratégia)
* SMA20: 20 dias (suavização maior)



In [ ]:
# CÁLCULO DAS MÉDIAS MÓVEIS POR TICKER

# Cópia de segurança do DataFrame original
df_mm = df_final.copy()

# Converte a coluna 'Data' para datetime (caso necessário)
df_mm['Data'] = pd.to_datetime(df_mm['Data'], format='%Y-%m-%d', errors='coerce')

# Ordena corretamente
df_mm = df_mm.sort_values(by=['Ticker', 'Data'])

# Cálculo das médias móveis por ticker
for window in [5, 10, 15, 20]:
    nome_coluna = f'SMA{window}'
    df_mm[nome_coluna] = (
        df_mm.groupby('Ticker')['Close']
        .transform(lambda x: x.rolling(window=window, min_periods=1).mean())
    )

# Verificação
print(f" Médias móveis calculadas com sucesso para {df_mm['Ticker'].nunique()} ativos.")
display(data_table.DataTable(df_mm.tail(10)))


In [ ]:
print("Colunas disponíveis em df_diario:")
print(df_diario.columns.tolist())

In [ ]:
print("Colunas disponíveis em dados_formatado:")
print(dados_formatado.columns.tolist())

In [ ]:
import pandas as pd
from datetime import date, timedelta

# --- 1. Geração de todas as datas no intervalo ---
inicio = date(2025, 1, 1)
fim = date(2025, 5, 4)
todas_datas = pd.date_range(start=inicio, end=fim, freq='D').date

# --- 2. Tickers disponíveis ---
tickers = dados_formatado['ticker'].unique().tolist()

# --- 3. Produto cartesiano: todos os dias para cada ticker ---
base_completa = pd.MultiIndex.from_product(
    [todas_datas, tickers],
    names=['date', 'ticker']
).to_frame(index=False)

# --- 4. Junção com base de preços ---
df_precos = dados_formatado.copy()
df_precos['date'] = pd.to_datetime(df_precos['date']).dt.date
merged_1 = pd.merge(base_completa, df_precos, how='left', on=['date', 'ticker'])

# --- 5. Junção com base de sentimentos ---
df_diario['date'] = pd.to_datetime(df_diario['date']).dt.date
merged_2 = pd.merge(merged_1, df_diario, how='left', on=['date', 'ticker'])

# --- 6. Atualização de Dia da Semana com nomes em português ---
dias_pt = {
    'Monday': 'Segunda-feira',
    'Tuesday': 'Terça-feira',
    'Wednesday': 'Quarta-feira',
    'Thursday': 'Quinta-feira',
    'Friday': 'Sexta-feira',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}
merged_2['Dia_da_semana'] = pd.to_datetime(merged_2['date']).dt.day_name().map(dias_pt)

# --- 7. Organização final ---
merged_2 = merged_2.sort_values(by=['ticker', 'date']).reset_index(drop=True)

# --- 8. Estatísticas ---
print("✅ Base unificada (preço + sentimento) criada com todas as datas.")
print(f"Total de linhas: {merged_2.shape[0]}")
print(f"Período coberto: {merged_2['date'].min()} até {merged_2['date'].max()}")
print(f"Colunas disponíveis: {merged_2.columns.tolist()}")

# --- 9. Visualização ---
display(merged_2.head(5))

# --- 10. Salvamento final ---
merged_2.to_csv("data/BaseCompleta_2025.csv", sep='|', index=False)
merged_2.to_excel("data/BaseCompleta_2025.xlsx", index=False)


In [ ]:
import backtrader as bt

class SentimentStrat(bt.Strategy):
    params = (
        ('sentiment_dict', None),
        ('sma_col', 'SMA15')
    )

    def __init__(self):
        self.order = None
        self.dataclose = self.datas[0].close
        self.dates = self.datas[0].datetime

    def next(self):
        current_date = bt.num2date(self.dates[0]).date()
        sentiment_today = self.params.sentiment_dict.get(current_date)
        sentiment_yesterday = self.params.sentiment_dict.get(current_date - pd.Timedelta(days=1))

        if sentiment_today is None or sentiment_yesterday is None:
            return  # Não há dado de sentimento suficiente

        delta_sentiment = sentiment_today - sentiment_yesterday
        sma_value = getattr(self.datas[0], self.params.sma_col, None)[0]

        if sma_value is None:
            return  # Não há SMA suficiente

        if not self.position:
            if delta_sentiment > 0.5 and self.dataclose[0] > sma_value:
                self.order = self.buy()
        else:
            if delta_sentiment < -0.5 and self.dataclose[0] < sma_value:
                self.order = self.sell()


In [ ]:
import yfinance as yf
import backtrader as bt
import pandas as pd


class SentimentStrat(bt.Strategy):
    params = (
        ('sentiment_dict', {}),
        ('sma_col', 'sma'),
    )

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.sentiment = self.p.sentiment_dict
        self.sma = self.datas[0].sma

    def next(self):
        data_atual = self.datas[0].datetime.date(0)

        if data_atual not in self.sentiment:
            return

        sentimento_hoje = self.sentiment.get(data_atual, 0)
        sentimento_ontem = self.sentiment.get(
            data_atual - pd.Timedelta(days=1), 0)
        delta_sent = sentimento_hoje - sentimento_ontem

        if self.order:
            return

        if not self.position:
            if delta_sent > 0.5 and self.dataclose[0] > self.sma[0]:
                self.order = self.buy(size=100)
        else:
            if delta_sent < -0.5 and self.dataclose[0] < self.sma[0]:
                self.order = self.sell(size=100)


class PandasSentiment(bt.feeds.PandasData):
    lines = ('sma',)
    params = (
        ('datetime', None),
        ('open', 'Open'),
        ('high', 'High'),
        ('low', 'Low'),
        ('close', 'Close'),
        ('volume', 'Volume'),
        ('openinterest', -1),
        ('sma', -1),  # -1 = procura uma coluna com esse nome
    )


def run_strategy(ticker, start, end, sentiment_series, sma_col='SMA15', plot=False):
    print(f"Executando estratégia para: {ticker} | SMA: {sma_col}")

    janela = int(sma_col.replace('SMA', ''))

    df_price = yf.download(ticker, start=start, end=end)
    if df_price.empty:
        print(f"[ERRO] Dados de preço vazios para {ticker}.")
        return None

   # **Change 1:** Check if columns is MultiIndex, and if so, convert to single level
    if isinstance(df_price.columns, pd.MultiIndex):
        df_price.columns = df_price.columns.get_level_values(0)

    df_price['sma'] = df_price['Close'].rolling(window=janela).mean()
    df_price = df_price[['Open', 'High', 'Low', 'Close', 'Volume', 'sma']].dropna()
    df_price.index.name = 'datetime'

    # **Change 2 (removed):** Removed converting column names to strings as it's unnecessary after flattening the MultiIndex

    data_bt = PandasSentiment(dataname=df_price)

    cerebro = bt.Cerebro()
    cerebro.adddata(data_bt)
    cerebro.addstrategy(SentimentStrat, sentiment_dict=sentiment_series)
    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=100)

    valor_inicial = cerebro.broker.getvalue()
    cerebro.run()
    valor_final = cerebro.broker.getvalue()

    if plot:
        cerebro.plot(volume=False, iplot=False)

    return {
        'ticker': ticker,
        'sma': sma_col,
        'valor_inicial': valor_inicial,
        'valor_final': valor_final,
        'lucro': valor_final - valor_inicial,
        'lucro_pct': (valor_final - valor_inicial) / valor_inicial * 100
    }


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
from collections import defaultdict

# === PARÂMETROS ===
ticker = 'VALE3.SA'
modelos_sentimento = [
    'sentiment_lex',
    'sentiment_LSTM',
    'sentiment_FinBERT_PT_BR',
    'sentiment_FinBERT_Turing',
    'sentiment_Multilingual_Sentiment',
    'sentiment_random'  # modelo de benchmark aleatório
]
capital_inicial = 100_000
taxa_b3 = 0.0003  # 0,03%

# === PERÍODO ===
datas_validas = df_diario[df_diario['ticker'] == ticker]['date'].dropna()
inicio = pd.to_datetime(datas_validas.min()).strftime('%Y-%m-%d')
fim    = pd.to_datetime(datas_validas.max()).strftime('%Y-%m-%d')
print(f"📅 Período: {inicio} a {fim}")

# === PREÇOS ===
df_price = yf.download(ticker, start=inicio, end=fim, progress=False)
if isinstance(df_price.columns, pd.MultiIndex):
    df_price.columns = ['_'.join(col).strip() if col[1] else col[0] for col in df_price.columns]
df_price = df_price.reset_index()
df_price['date'] = pd.to_datetime(df_price['Date']).dt.date
col_close = f'Close_{ticker}' if f'Close_{ticker}' in df_price.columns else 'Close'
df_price['close'] = df_price[col_close]

# === DIRETÓRIO ===
os.makedirs("data", exist_ok=True)

# === RESULTADOS GERAIS ===
resultados = []
curvas = {}

for modelo_sent in modelos_sentimento:
    print(f"\n▶️ Modelo proporcional: {modelo_sent}")

    if modelo_sent == 'sentiment_random':
        # Gera valores aleatórios entre -1 e 1
        df_sent = pd.DataFrame({
            'date': df_price['date'],
            'sentiment_random': np.random.uniform(-1, 1, size=len(df_price))
        })
    else:
        df_sent = df_diario[df_diario['ticker'] == ticker][['date', modelo_sent]].dropna()
        if df_sent.empty:
            print(f"⚠️ Sem dados para {modelo_sent}. Pulando.")
            continue

    df_sent['date'] = pd.to_datetime(df_sent['date']).dt.date
    sentimento_dict = df_sent.set_index('date')[modelo_sent].to_dict()

    capital = capital_inicial
    posicao = 0
    decisoes = []
    operacoes = 0
    acertos = 0
    taxas_B3 = 0.0
    IR_pago = 0.0
    historico_valor = []

    vendas_mes = defaultdict(float)
    lucro_mes = defaultdict(float)
    prejuizo_acumulado = 0.0
    preco_medio = 0.0

    for _, row in df_price.iterrows():
        data = pd.to_datetime(row['date']).date()
        preco = row['close']
        sentimento = sentimento_dict.get(data, 0.0)
        acao = "MANTER"
        quantidade = 0
        taxa = 0.0

        # --- COMPRA ---
        if sentimento > 0:
            valor_para_compra = capital * sentimento
            quantidade = int(valor_para_compra / preco)
            if quantidade > 0:
                valor_op = quantidade * preco
                taxa = valor_op * taxa_b3
                custo_total = valor_op + taxa
                if capital >= custo_total:
                    preco_medio = ((preco_medio * posicao) + valor_op) / (posicao + quantidade) if posicao > 0 else preco
                    capital -= custo_total
                    posicao += quantidade
                    taxas_B3 += taxa
                    operacoes += 1
                    acao = f"COMPRA {quantidade}"

        # --- VENDA ---
        elif sentimento < 0 and posicao > 0:
            quantidade = int(posicao * abs(sentimento))
            if quantidade > 0:
                valor_op = quantidade * preco
                taxa = valor_op * taxa_b3
                receita_liquida = valor_op - taxa
                capital += receita_liquida
                posicao -= quantidade
                taxas_B3 += taxa
                operacoes += 1
                acao = f"VENDA {quantidade}"

                lucro_real = (preco - preco_medio) * quantidade
                mes = data.strftime('%Y-%m')
                vendas_mes[mes] += valor_op

                if lucro_real > 0:
                    acertos += 1
                    lucro_mes[mes] += lucro_real
                else:
                    prejuizo_acumulado += abs(lucro_real)

        valor_total = capital + posicao * preco
        historico_valor.append(valor_total)

        decisoes.append({
            'date': data,
            'modelo': modelo_sent,
            'sentimento': sentimento,
            'close': preco,
            'acao': acao,
            'quantidade': quantidade,
            'posicao': posicao,
            'capital_em_dinheiro': capital,
            'capital_total_estimado': valor_total,
            'taxas_B3': taxas_B3
        })

    # === CÁLCULO DO IR ===
    for mes in lucro_mes:
        lucro = lucro_mes[mes] - prejuizo_acumulado
        if lucro <= 0:
            prejuizo_acumulado = abs(lucro)
            continue
        if vendas_mes[mes] > 20_000:
            IR_pago += lucro * 0.15
            prejuizo_acumulado = 0.0

    # === MÉTRICAS ===
    lucro_bruto = valor_total - capital_inicial
    lucro_liquido = lucro_bruto - IR_pago
    lucro_pct = (lucro_liquido / capital_inicial) * 100
    retorno_diario = pd.Series(historico_valor).pct_change().dropna()
    sharpe = retorno_diario.mean() / retorno_diario.std() * np.sqrt(252) if len(retorno_diario) > 1 else 0
    max_drawdown = (pd.Series(historico_valor).cummax() - pd.Series(historico_valor)).max()
    taxa_acerto = acertos / operacoes * 100 if operacoes > 0 else 0

    curvas[modelo_sent] = pd.DataFrame(decisoes)

    resultados.append({
        'modelo': modelo_sent,
        'valor_final': valor_total,
        'lucro_bruto': lucro_bruto,
        'IR_pago': IR_pago,
        'lucro_liquido': lucro_liquido,
        'lucro_pct': lucro_pct,
        'n_operacoes': operacoes,
        'taxa_acerto_pct': taxa_acerto,
        'sharpe_ratio': sharpe,
        'max_drawdown': max_drawdown,
        'taxas_B3': taxas_B3
    })

# === RESUMO ===
df_resultados = pd.DataFrame(resultados).sort_values(by='lucro_pct', ascending=False)
print("\n📊 COMPARATIVO COM MODELO ALEATÓRIO (Random Benchmark):")
display(df_resultados)

# === PLOTAGEM ===
plt.figure(figsize=(14, 7))
for modelo, df in curvas.items():
    df['date'] = pd.to_datetime(df['date'])
    lucro = df_resultados[df_resultados['modelo'] == modelo]['lucro_pct'].values[0]
    plt.plot(df['date'], df['capital_total_estimado'], label=f"{modelo} ({lucro:.2f}%)")

plt.title(f"Evolução da Carteira – Comparação com Benchmark Aleatório ({ticker})", fontsize=14, fontweight='bold')
plt.xlabel("Data")
plt.ylabel("Capital Total (R$)")
plt.grid(True, linestyle='--', alpha=0.5)
plt.xticks(rotation=45)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.tight_layout()
plt.legend(title='Modelo (Lucro Líquido %)', fontsize=10)
plt.show()


In [ ]:
# === IMPORTAÇÕES ===
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
from collections import defaultdict

# === PARÂMETROS ===
ticker = 'ITUB4.SA'
modelos_sentimento = [
    'sentiment_lex',
    'sentiment_LSTM',
    'sentiment_FinBERT_PT_BR',
    'sentiment_FinBERT_Turing',
    'sentiment_Multilingual_Sentiment',
    'sentiment_random'  # Benchmark aleatório corrigido
]
capital_inicial = 100_000
taxa_b3 = 0.0003

# === PERÍODO ===
datas_validas = df_diario[df_diario['ticker'] == ticker]['date'].dropna()
inicio = pd.to_datetime(datas_validas.min()).strftime('%Y-%m-%d')
fim = pd.to_datetime(datas_validas.max()).strftime('%Y-%m-%d')
print(f"📅 Período: {inicio} a {fim}")

# === PREÇOS ===
df_price = yf.download(ticker, start=inicio, end=fim, progress=False)
if isinstance(df_price.columns, pd.MultiIndex):
    df_price.columns = ['_'.join(col).strip() if col[1] else col[0] for col in df_price.columns]
df_price = df_price.reset_index()
df_price['date'] = pd.to_datetime(df_price['Date']).dt.date
df_price['close'] = df_price[f'Close_{ticker}'] if f'Close_{ticker}' in df_price.columns else df_price['Close']

# === DIRETÓRIO ===
os.makedirs("data", exist_ok=True)

# === RESULTADOS ===
resultados = []
curvas = {}

for modelo_sent in modelos_sentimento:
    print(f"\n▶️ Simulando modelo: {modelo_sent}")

    if modelo_sent == 'sentiment_random':
        # Geração de sentimento aleatório com variabilidade autêntica
        np.random.seed(None)  # garante aleatoriedade diferente em cada execução
        ruido = np.random.normal(loc=0, scale=0.6, size=len(df_price))
        ruido = np.clip(ruido, -1, 1)
        df_sent = pd.DataFrame({
            'date': df_price['date'],
            'sentiment_random': ruido
        })
    else:
        df_sent = df_diario[df_diario['ticker'] == ticker][['date', modelo_sent]].dropna()
        if df_sent.empty:
            print(f"⚠️ Sem dados para {modelo_sent}. Pulando.")
            continue

    df_sent['date'] = pd.to_datetime(df_sent['date']).dt.date
    sentimento_dict = df_sent.set_index('date')[modelo_sent].to_dict()

    capital = capital_inicial
    posicao = 0
    preco_medio = 0.0
    decisoes = []
    operacoes = 0
    acertos = 0
    taxas_B3 = 0.0
    IR_pago = 0.0
    historico_valor = []

    vendas_mes = defaultdict(float)
    lucro_mes = defaultdict(float)
    prejuizo_acumulado = 0.0

    for _, row in df_price.iterrows():
        data = pd.to_datetime(row['date']).date()
        preco = row['close']
        sentimento = sentimento_dict.get(data, 0.0)

        acao = "MANTER"
        quantidade = 0
        taxa = 0.0

        if sentimento > 0:
            valor_para_compra = capital * sentimento
            quantidade = int(valor_para_compra / preco)
            if quantidade > 0:
                valor_op = quantidade * preco
                taxa = valor_op * taxa_b3
                custo_total = valor_op + taxa
                if capital >= custo_total:
                    preco_medio = ((preco_medio * posicao) + valor_op) / (posicao + quantidade) if posicao > 0 else preco
                    capital -= custo_total
                    posicao += quantidade
                    taxas_B3 += taxa
                    operacoes += 1
                    acao = f"COMPRA {quantidade}"

        elif sentimento < 0 and posicao > 0:
            quantidade = int(posicao * abs(sentimento))
            if quantidade > 0:
                valor_op = quantidade * preco
                taxa = valor_op * taxa_b3
                receita_liquida = valor_op - taxa
                capital += receita_liquida
                posicao -= quantidade
                taxas_B3 += taxa
                operacoes += 1
                acao = f"VENDA {quantidade}"

                lucro_real = (preco - preco_medio) * quantidade
                mes = data.strftime('%Y-%m')
                vendas_mes[mes] += valor_op

                if lucro_real > 0:
                    acertos += 1
                    lucro_mes[mes] += lucro_real
                else:
                    prejuizo_acumulado += abs(lucro_real)

        valor_total = capital + posicao * preco
        historico_valor.append(valor_total)

        decisoes.append({
            'date': data,
            'modelo': modelo_sent,
            'sentimento': sentimento,
            'close': preco,
            'acao': acao,
            'quantidade': quantidade,
            'posicao': posicao,
            'capital_em_dinheiro': capital,
            'capital_total_estimado': valor_total,
            'taxas_B3': taxas_B3
        })

    # === CÁLCULO DO IR ===
    for mes in lucro_mes:
        lucro = lucro_mes[mes] - prejuizo_acumulado
        if lucro <= 0:
            prejuizo_acumulado = abs(lucro)
            continue
        if vendas_mes[mes] > 20_000:
            IR_pago += lucro * 0.15
            prejuizo_acumulado = 0.0

    lucro_bruto = valor_total - capital_inicial
    lucro_liquido = lucro_bruto - IR_pago
    lucro_pct = (lucro_liquido / capital_inicial) * 100
    retorno_diario = pd.Series(historico_valor).pct_change().dropna()
    sharpe = retorno_diario.mean() / retorno_diario.std() * np.sqrt(252) if len(retorno_diario) > 1 else 0
    max_drawdown = (pd.Series(historico_valor).cummax() - pd.Series(historico_valor)).max()
    taxa_acerto = acertos / operacoes * 100 if operacoes > 0 else 0

    curvas[modelo_sent] = pd.DataFrame(decisoes)

    resultados.append({
        'modelo': modelo_sent,
        'valor_final': valor_total,
        'lucro_bruto': lucro_bruto,
        'IR_pago': IR_pago,
        'lucro_liquido': lucro_liquido,
        'lucro_pct': lucro_pct,
        'n_operacoes': operacoes,
        'taxa_acerto_pct': taxa_acerto,
        'sharpe_ratio': sharpe,
        'max_drawdown': max_drawdown,
        'taxas_B3': taxas_B3
    })

# === RESUMO FINAL ===
df_resultados = pd.DataFrame(resultados).sort_values(by='lucro_pct', ascending=False)
print("\n📊 COMPARATIVO COM BENCHMARK ALEATÓRIO:")
display(df_resultados)

# === PLOTAGEM ===
plt.figure(figsize=(14, 7))
for modelo, df in curvas.items():
    df['date'] = pd.to_datetime(df['date'])
    lucro = df_resultados[df_resultados['modelo'] == modelo]['lucro_pct'].values[0]
    plt.plot(df['date'], df['capital_total_estimado'], label=f"{modelo} ({lucro:.2f}%)")

plt.title(f"Evolução da Carteira – Benchmark Aleatório ({ticker})", fontsize=14, fontweight='bold')
plt.xlabel("Data")
plt.ylabel("Capital Total (R$)")
plt.grid(True, linestyle='--', alpha=0.5)
plt.xticks(rotation=45)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.tight_layout()
plt.legend(title='Modelo (Lucro Líquido %)', fontsize=10)
plt.show()


<a id='5.2'></a>
## 5.2. Results for Individual Stocks

First running the strategy for google

In [ ]:
# === IMPORTAÇÕES ===
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
from collections import defaultdict

# === PARÂMETROS ===
ticker = 'VALE3.SA'
modelos_sentimento = [
    'sentiment_lex',
    'sentiment_LSTM',
    'sentiment_FinBERT_PT_BR',
    'sentiment_FinBERT_Turing',
    'sentiment_Multilingual_Sentiment',
    'sentiment_random'
]
capital_inicial = 100_000
taxa_b3 = 0.0003  # 0,03%
loss_aversion_ratio = 2.25  # fator de aversão à perda

# === PERÍODO ===
datas_validas = df_diario[df_diario['ticker'] == ticker]['date'].dropna()
inicio = pd.to_datetime(datas_validas.min()).strftime('%Y-%m-%d')
fim = pd.to_datetime(datas_validas.max()).strftime('%Y-%m-%d')
print(f"📅 Período: {inicio} a {fim}")

# === PREÇOS ===
df_price = yf.download(ticker, start=inicio, end=fim, progress=False)
if isinstance(df_price.columns, pd.MultiIndex):
    df_price.columns = ['_'.join(col).strip() if col[1] else col[0] for col in df_price.columns]
df_price = df_price.reset_index()
df_price['date'] = pd.to_datetime(df_price['Date']).dt.date
col_close = f'Close_{ticker}' if f'Close_{ticker}' in df_price.columns else 'Close'
df_price['close'] = df_price[col_close]

# === DIRETÓRIO ===
os.makedirs("data", exist_ok=True)

# === RESULTADOS GERAIS ===
resultados = []
curvas = {}

for modelo_sent in modelos_sentimento:
    print(f"\n▶️ Modelo proporcional: {modelo_sent}")

    if modelo_sent == 'sentiment_random':
        df_sent = pd.DataFrame({
            'date': df_price['date'],
            'sentiment_random': np.random.uniform(-1, 1, size=len(df_price))
        })
    else:
        df_sent = df_diario[df_diario['ticker'] == ticker][['date', modelo_sent]].dropna()
        if df_sent.empty:
            print(f"⚠️ Sem dados para {modelo_sent}. Pulando.")
            continue

    df_sent['date'] = pd.to_datetime(df_sent['date']).dt.date
    sentimento_dict = df_sent.set_index('date')[modelo_sent].to_dict()

    capital = capital_inicial
    posicao = 0
    preco_medio = 0.0
    decisoes = []
    operacoes = 0
    acertos = 0
    taxas_B3 = 0.0
    IR_pago = 0.0
    historico_valor = []

    vendas_mes = defaultdict(float)
    lucro_mes = defaultdict(float)
    prejuizo_acumulado = 0.0

    for _, row in df_price.iterrows():
        data = pd.to_datetime(row['date']).date()
        preco = row['close']
        sentimento = sentimento_dict.get(data, 0.0)

        # Ajuste com aversão à perda
        peso = sentimento if sentimento >= 0 else sentimento * loss_aversion_ratio

        acao = "MANTER"
        quantidade = 0
        taxa = 0.0

        # --- COMPRA proporcional ---
        if peso > 0:
            valor_para_compra = capital * peso
            quantidade = int(valor_para_compra / preco)
            if quantidade > 0:
                valor_op = quantidade * preco
                taxa = valor_op * taxa_b3
                custo_total = valor_op + taxa
                if capital >= custo_total:
                    preco_medio = ((preco_medio * posicao) + valor_op) / (posicao + quantidade) if posicao > 0 else preco
                    capital -= custo_total
                    posicao += quantidade
                    taxas_B3 += taxa
                    operacoes += 1
                    acao = f"COMPRA {quantidade}"

        # --- VENDA proporcional ---
        elif peso < 0 and posicao > 0:
            quantidade = int(posicao * abs(peso))
            if quantidade > 0:
                valor_op = quantidade * preco
                taxa = valor_op * taxa_b3
                receita_liquida = valor_op - taxa
                capital += receita_liquida
                posicao -= quantidade
                taxas_B3 += taxa
                operacoes += 1
                acao = f"VENDA {quantidade}"

                lucro_real = (preco - preco_medio) * quantidade
                mes = data.strftime('%Y-%m')
                vendas_mes[mes] += valor_op

                if lucro_real > 0:
                    acertos += 1
                    lucro_mes[mes] += lucro_real
                else:
                    prejuizo_acumulado += abs(lucro_real)

        valor_total = capital + posicao * preco
        historico_valor.append(valor_total)

        decisoes.append({
            'date': data,
            'modelo': modelo_sent,
            'sentimento': sentimento,
            'close': preco,
            'acao': acao,
            'quantidade': quantidade,
            'posicao': posicao,
            'capital_em_dinheiro': capital,
            'capital_total_estimado': valor_total,
            'taxas_B3': taxas_B3
        })

    # === CÁLCULO DE IR ===
    for mes in lucro_mes:
        lucro = lucro_mes[mes] - prejuizo_acumulado
        if lucro <= 0:
            prejuizo_acumulado = abs(lucro)
            continue
        if vendas_mes[mes] > 20_000:
            IR_pago += lucro * 0.15
            prejuizo_acumulado = 0.0

    # === MÉTRICAS FINAIS ===
    lucro_bruto = valor_total - capital_inicial
    lucro_liquido = lucro_bruto - IR_pago
    lucro_pct = (lucro_liquido / capital_inicial) * 100
    retorno_diario = pd.Series(historico_valor).pct_change().dropna()
    sharpe = retorno_diario.mean() / retorno_diario.std() * np.sqrt(252) if len(retorno_diario) > 1 else 0
    max_drawdown = (pd.Series(historico_valor).cummax() - pd.Series(historico_valor)).max()
    taxa_acerto = acertos / operacoes * 100 if operacoes > 0 else 0

    curvas[modelo_sent] = pd.DataFrame(decisoes)

    resultados.append({
        'modelo': modelo_sent,
        'valor_final': valor_total,
        'lucro_bruto': lucro_bruto,
        'IR_pago': IR_pago,
        'lucro_liquido': lucro_liquido,
        'lucro_pct': lucro_pct,
        'n_operacoes': operacoes,
        'taxa_acerto_pct': taxa_acerto,
        'sharpe_ratio': sharpe,
        'max_drawdown': max_drawdown,
        'taxas_B3': taxas_B3
    })

# === RESUMO FINAL ===
df_resultados = pd.DataFrame(resultados).sort_values(by='lucro_pct', ascending=False)
print("\n📊 RESUMO FINAL COM APLICAÇÃO DE AVERSÃO À PERDA:")
display(df_resultados)

# === PLOTAGEM ===
plt.figure(figsize=(14, 7))
for modelo, df in curvas.items():
    df['date'] = pd.to_datetime(df['date'])
    lucro = df_resultados[df_resultados['modelo'] == modelo]['lucro_pct'].values[0]
    plt.plot(df['date'], df['capital_total_estimado'], label=f"{modelo} ({lucro:.2f}%)")

plt.title(f"Evolução da Carteira – Aversão à Perda e Benchmark Aleatório ({ticker})", fontsize=14, fontweight='bold')
plt.xlabel("Data")
plt.ylabel("Capital Total (R$)")
plt.grid(True, linestyle='--', alpha=0.5)
plt.xticks(rotation=45)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.tight_layout()
plt.legend(title='Modelo (Lucro Líquido %)', fontsize=10)
plt.show()


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from collections import defaultdict

# === PARÂMETROS ===
tickers = ['VALE3.SA', 'PETR4.SA', 'ITUB4.SA', 'BBAS3.SA']
modelos_sent = [
    'sentiment_lex',
    'sentiment_LSTM',
    'sentiment_FinBERT_PT_BR',
    'sentiment_FinBERT_Turing',
    'sentiment_Multilingual_Sentiment',
    'sentiment_random'
]
capital_inicial = 100_000
taxa_b3 = 0.0003
start = '2025-01-01'
end   = '2025-05-04'

# Estruturas para armazenar curvas
curvas_hold  = {}
curvas_trade = {}

for ticker in tickers:
    # 1. Obtenção de preços
    df_price = yf.download(ticker, start=start, end=end, progress=False).reset_index()
    df_price['date']  = pd.to_datetime(df_price['Date']).dt.date
    # normalização de coluna de fechamento
    close_col = f'Close_{ticker}'
    df_price['close'] = (df_price[close_col] if close_col in df_price else df_price['Close'])

    # 2. Cálculo buy-and-hold
    preco0 = df_price.loc[0, 'close']
    n_acoes = int(capital_inicial / preco0)
    sobra    = capital_inicial - n_acoes * preco0
    df_hold = df_price[['date']].copy()
    df_hold['capital_hold'] = n_acoes * df_price['close'] + sobra
    curvas_hold[ticker] = df_hold

    # 3. Simulação de trading
    # (pressupõe existência de df_diario com sentimentos para todos os tickers)
    df_sent_all = df_diario[df_diario['ticker']==ticker]
    capital = capital_inicial
    posicao = 0
    preco_medio = 0.0
    historico = []
    # geração de sentimento aleatório
    for modelo in modelos_sent:
        if modelo=='sentiment_random':
            ruido = np.random.normal(0,0.6, size=len(df_price))
            ruido = np.clip(ruido, -1, 1)
            df_sent = pd.DataFrame({'date':df_price['date'],'sentiment_random':ruido})
        else:
            df_tmp = df_sent_all[['date',modelo]].dropna()
            df_sent = df_tmp.copy()
        sentimento_dict = df_sent.set_index(pd.to_datetime(df_sent['date']).dt.date)[modelo].to_dict()
        capital = capital_inicial; posicao=0; preco_medio=0.0; taxas_B3=0.0
        historico = []
        for row in df_price.itertuples():
            d = row.date; p = row.close
            s = sentimento_dict.get(d, 0.0)
            # compra
            if s>0:
                v = capital * s
                q = int(v/p)
                if q>0:
                    op = q*p; tx = op*taxa_b3
                    if capital>=op+tx:
                        preco_medio = ((preco_medio*posicao)+op)/(posicao+q) if posicao>0 else p
                        capital -= op+tx
                        posicao += q
                        taxas_B3 += tx
            # venda
            elif s<0 and posicao>0:
                q = int(posicao * abs(s))
                if q>0:
                    op = q*p; tx = op*taxa_b3
                    capital += op-tx
                    posicao -= q
                    taxas_B3 += tx
            total = capital + posicao*p
            historico.append({'date':d, 'capital': total})
        curvas_trade.setdefault(ticker, pd.DataFrame(historico))

# 4. Plotagem comparativa em subplots
n = len(tickers)
fig, axes = plt.subplots(n, 1, figsize=(12, 4*n), sharex=True)
for ax, ticker in zip(axes, tickers):
    df_h = curvas_hold[ticker]
    df_t = curvas_trade[ticker]
    df_h['date'] = pd.to_datetime(df_h['date'])
    df_t['date'] = pd.to_datetime(df_t['date'])
    ax.plot(df_h['date'], df_h['capital_hold'],
            '--', color='gray', label='Buy-and-Hold')
    ax.plot(df_t['date'], df_t['capital'],
            linewidth=1.5, label='Trading')
    ax.set_title(f'{ticker}', fontweight='bold')
    ax.grid(linestyle=':', alpha=0.6)
    ax.legend()
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    for label in ax.get_xticklabels():
        label.set_rotation(45)
plt.xlabel('Data')
plt.ylabel('Capital Total (R$)')
plt.tight_layout()
plt.show()
